# Assignment 2 – Comparative Financial QA System: RAG vs Fine-Tuning

# Group 58:


# 1. Amit Kumar (2023ac05242)

# 2. Abhansi Sharma (2023ac05988)

# 3. Jagtab Charudatta Prashant (2023ad05065)

# 4. Pokuri Abhinav (2023ad05090)

# 5. Tribhuwan Anudit Robin (2023ac05094)

# Install & Imports

**What this does:** Installs (if needed) and imports core libraries for data prep, tokenization, dense/sparse retrieval, cross-encoder re-ranking, and SimFin access.

**Why it matters:** Even if you only use SimFin CSVs, downstream **RAG** relies on `transformers`, `sentence-transformers`, `faiss` (dense index), and `scikit-learn` (TF-IDF). Keep optional parsers commented unless you need them.


In [1]:
# --- Section: 1.1) Install & Imports ---
# If first run on Colab, uncomment the pip lines:

import os, re, json, time, math, glob, unicodedata, subprocess, sys, textwrap, warnings, uuid, random
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional, Any

import numpy as np
import pandas as pd
import torch
from statistics import mean
import datetime as dt

# Dense retrieval & encoding
import faiss
from sentence_transformers import SentenceTransformer, CrossEncoder
from torch.utils.data import Dataset, DataLoader
from torch import nn
import ipywidgets

# Sparse retrieval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Fuzzy matching
from difflib import SequenceMatcher
from rapidfuzz import fuzz, process

# SimFin API
import simfin as sf

# Transformers (tokenizer + CE)
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification


# import of counter
from collections import Counter


os.environ["TQDM_NOTEBOOK"] = "0"   # tells tqdm to use console mode
warnings.filterwarnings("ignore")


print("[ok] imports ready")


C:\Users\91948\Downloads\Conv AI assignment 2\conv2\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


[ok] imports ready


# 1. Data Collection & Preprocessing

# 1.2) Configure SimFin & Years of Interest

**Purpose:** Centralize config (API key, market, statements, years) so the whole pipeline is reproducible.

**Key idea:** Keep `YEARS_REQUIRED` in one place to align all filters and evaluation later.


In [2]:
# --- Put your SimFin API key here (we'll also read SIMFIN_API_KEY if set) ---
SIMFIN_API_KEY = os.getenv("SIMFIN_API_KEY") or "e0800424-7681-4761-a514-031950ce6450"  # <-- your key

sf.set_api_key(SIMFIN_API_KEY)
sf.set_data_dir("simfin_data")

# Masked print
mask = SIMFIN_API_KEY[:4] + "…" + SIMFIN_API_KEY[-4:] if len(SIMFIN_API_KEY) > 8 else "****"
print(f"[ok] SimFin configured ({mask})")

# Target years for PoC: ensure both are available for chosen companies
YEARS_REQUIRED = {2023, 2024}
print("[info] will require all of:", sorted(YEARS_REQUIRED))


[ok] SimFin configured (e080…6450)
[info] will require all of: [2023, 2024]


# 1.3) Load SimFin Annual Statements (US) & Helper Functions

**What happens:** Load annual IS/BS/CF and provide helpers for coverage and tidy reshape.

**Why this structure:** A single “reader” per type plus small utilities makes later RAG and evaluation simpler.


In [3]:
# Load annual statements for US market
income  = sf.load_income(variant='annual', market='us')      # Income Statement
balance = sf.load_balance(variant='annual', market='us')     # Balance Sheet
cashfl  = sf.load_cashflow(variant='annual', market='us')    # Cash Flow

def _col(df: pd.DataFrame, name: str, aliases: tuple=()) -> str:
    m = {c.lower(): c for c in df.columns}
    for k in (name, *aliases):
        if k.lower() in m:
            return m[k.lower()]
    raise KeyError(f"Column '{name}' not found; first cols: {list(df.columns)[:6]}")

def tickers_with_years(df: pd.DataFrame, years={2023, 2024}) -> set:
    """Tickers that have all target years present in this statement frame."""
    x = df.reset_index() if isinstance(df.index, pd.MultiIndex) else df
    c_t, c_y = _col(x, "Ticker"), _col(x, "Fiscal Year", aliases=("FiscalYear","FY"))
    y = x[[c_t, c_y]].dropna().drop_duplicates()
    y = y[y[c_y].isin(years)]
    counts = y.groupby(c_t)[c_y].nunique()
    return set(counts[counts >= len(years)].index)

def filter_last2_fy(df: pd.DataFrame, tickers: List[str]) -> pd.DataFrame:
    """Keep only selected tickers and their latest 2 fiscal years (per ticker)."""
    x = df.reset_index() if isinstance(df.index, pd.MultiIndex) else df
    c_t, c_y = _col(x, "Ticker"), _col(x, "Fiscal Year", aliases=("FiscalYear","FY"))
    sub = x[x[c_t].isin(tickers)].copy()
    sub["__rank__"] = sub.groupby(c_t)[c_y].rank(ascending=False, method="dense")
    return sub[sub["__rank__"] <= 2].drop(columns="__rank__").reset_index(drop=True)

def tidy_simfin(df: pd.DataFrame, statement: str) -> pd.DataFrame:
    """Wide → long: company,ticker,statement,fiscal_year,line_item,value,unit"""
    x = df.reset_index() if isinstance(df.index, pd.MultiIndex) else df
    c_t, c_y = _col(x, "Ticker"), _col(x, "Fiscal Year", aliases=("FiscalYear","FY"))
    c_co = next((c for c in x.columns if c.lower() in ("company name","companyname","name")), None)
    c_cur= next((c for c in x.columns if "currency" in c.lower()), None)

    id_cols = {c_t, c_y}
    if c_co: id_cols.add(c_co)
    if c_cur: id_cols.add(c_cur)

    value_cols = [c for c in x.columns if c not in id_cols]
    long = (x[list(id_cols)+value_cols]
            .melt(id_vars=list(id_cols), var_name="line_item", value_name="value"))

    long.rename(columns={
        c_t:"ticker", c_y:"fiscal_year",
        (c_co or "company"):"company",
        (c_cur or "unit"):"unit"
    }, inplace=True)

    if "company" not in long: long["company"] = long["ticker"]
    if "unit" not in long:    long["unit"] = "USD"

    long["value"] = pd.to_numeric(long["value"], errors="coerce")
    long = long.dropna(subset=["value"]).reset_index(drop=True)
    long["statement"] = statement
    return long[["company","ticker","statement","fiscal_year","line_item","value","unit"]]


Dataset "us-income-annual" on disk (2 days old).
- Loading from disk ... Done!
Dataset "us-balance-annual" on disk (2 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-annual" on disk (2 days old).
- Loading from disk ... Done!


# 1.4) Select Tickers with Full Coverage (2023 & 2024, IS/BS/CF)

**Rationale:** Ensure a consistent subset exists across IS/BS/CF for both years; later RAG and eval depend on this.

**Outcome:** A deterministic `TICKERS` list for all following steps.


In [4]:
yrs = YEARS_REQUIRED
is_ok = tickers_with_years(income,  yrs)
bs_ok = tickers_with_years(balance, yrs)
cf_ok = tickers_with_years(cashfl,  yrs)

eligible = sorted(list(is_ok & bs_ok & cf_ok))
print(f"[info] eligible tickers with IS/BS/CF for {sorted(yrs)} (showing up to 20):", eligible[:20])
print(f"[info] total eligible: {len(eligible)}")

# Choose a small set for your PoC (increase as desired)
MAX_TICKERS = 4
if len(eligible) == 0:
    raise RuntimeError("No eligible tickers found with full 2023 & 2024 coverage. Try broadening years or re-running later.")
TICKERS = eligible[:MAX_TICKERS]
print("[ok] using tickers:", TICKERS)


[info] eligible tickers with IS/BS/CF for [2023, 2024] (showing up to 20): ['AAGH', 'ADP', 'AEHR', 'AFRM', 'AI', 'AIT', 'AKTS', 'AMCR', 'AMRK', 'AMSWA', 'AMWD', 'ANEB', 'ANGO', 'AOSL', 'AOUT', 'APLD', 'ARAY', 'ASTC', 'ATGE', 'AVAV']
[info] total eligible: 186
[ok] using tickers: ['AAGH', 'ADP', 'AEHR', 'AFRM']


# 1.5) Build a Tidy Dataset (Selected Tickers × Years × IS/BS/CF) and Save

**Why tidy:** A single long dataframe makes it trivial to generate sections (passages) for RAG and gold values for evaluation.

**Side effect:** Writes `out/financials_last2y.csv` for easy inspection.


In [5]:
# Filter each financial statement down to the last two fiscal years for the requested tickers.
# Keeps structure identical; just narrows rows to recent years so downstream joins stay small & current.
income_2y  = filter_last2_fy(income,  TICKERS)
balance_2y = filter_last2_fy(balance, TICKERS)
cashfl_2y  = filter_last2_fy(cashfl,  TICKERS)

# Convert each (possibly wide) SimFin table into a consistent "tidy" long format:
# columns expected like: ticker | fiscal_year | statement | line_item | value
tidy_inc = tidy_simfin(income_2y,  "income_statement")
tidy_bal = tidy_simfin(balance_2y, "balance_sheet")
tidy_cfl = tidy_simfin(cashfl_2y,  "cash_flow")

# Union all three statements into one DataFrame for unified processing/analytics.
df_fin = pd.concat([tidy_inc, tidy_bal, tidy_cfl], ignore_index=True)

# Order rows for readability and deterministic downstream operations, then reset the integer index.
df_fin = df_fin.sort_values(["ticker","statement","fiscal_year","line_item"]).reset_index(drop=True)

# Ensure an output directory exists (no error if it already does), then persist the combined dataset to CSV.
os.makedirs("out", exist_ok=True)
df_fin.to_csv("out/financials_last2y.csv", index=False)

# Log a concise success message with row count and the sorted ticker list for quick verification.
print(f"[ok] wrote out/financials_last2y.csv | rows: {len(df_fin)} | tickers: {sorted(df_fin['ticker'].unique())}")

# In notebooks, evaluating this will display the first 12 rows to sanity-check the shape and content.
df_fin.head(12) 


[ok] wrote out/financials_last2y.csv | rows: 418 | tickers: ['AAGH', 'ADP', 'AEHR', 'AFRM']


,company,ticker,statement,fiscal_year,line_item,value,unit
0,AAGH,AAGH,balance_sheet,2023,"Cash, Cash Equivalents & Short Term Investments",5.415000e+04,USD
1,AAGH,AAGH,balance_sheet,2023,Inventories,1.083510e+05,USD
2,AAGH,AAGH,balance_sheet,2023,Long Term Debt,1.524788e+06,USD
3,AAGH,AAGH,balance_sheet,2023,Other Long Term Assets,5.375400e+04,USD
4,AAGH,AAGH,balance_sheet,2023,Payables & Accruals,1.640999e+06,USD
5,AAGH,AAGH,balance_sheet,2023,"Property, Plant & Equipment, Net",5.769200e+04,USD
6,AAGH,AAGH,balance_sheet,2023,Retained Earnings,-9.028110e+06,USD
7,AAGH,AAGH,balance_sheet,2023,Share Capital & Additional Paid-In Capital,4.732477e+06,USD
8,AAGH,AAGH,balance_sheet,2023,Shares (Basic),2.109340e+10,USD
9,AAGH,AAGH,balance_sheet,2023,Shares (Diluted),2.109340e+10,USD


# 1.6) Parsers for PDF / HTML / Excel (Optional)

**When to use:** Only if you bring your own filings. Each parser returns raw text; normalization happens next (§1.7).


In [6]:
# --- Optional imports for rich document parsing (PDF & HTML). ---
# These libraries are *optional*; the code will degrade gracefully if missing.
# - pdfplumber: high-quality text extraction from PDFs (layout-aware).
# - BeautifulSoup (bs4): HTML parsing and tag stripping.
try:
    import pdfplumber
    from bs4 import BeautifulSoup
except Exception:
    # If imports fail, set to None so downstream functions can raise
    # clear, actionable errors (or no-op) without crashing the notebook.
    pdfplumber = None
    BeautifulSoup = None


# --- PDF → plain text ---
# Reads a PDF file and concatenates text from each page.
# Notes:
# - Requires pdfplumber; raises a helpful RuntimeError if it's not installed.
# - Returns "" if the file path doesn't exist.
# - Skips pages that throw extraction errors instead of failing the whole run.
def extract_text_from_pdf(pdf_path: str) -> str:
    if pdfplumber is None:
        raise RuntimeError("pdfplumber not installed. Run: %pip install pdfplumber")
    if not os.path.exists(pdf_path): return ""
    out = []
    with pdfplumber.open(pdf_path) as pdf:
        for p in pdf.pages:
            try:
                t = p.extract_text() or ""  # ensure a string even if extraction returns None
                out.append(t)
            except Exception:
                continue  # be robust to page-level failures
    return "\n\n".join(out).strip()  # double-newline between pages; trim ends


# --- Excel (.xlsx) → plain text ---
# Flattens all sheets into a single string.
# Strategy:
# - Uses pandas.ExcelFile for efficient multi-sheet handling.
# - For each sheet, converts the DataFrame to a readable table string (no index),
#   and prefixes with a [Sheet: <name>] header for provenance.
# - Returns "" on any parsing error to avoid breaking the pipeline.
def extract_text_from_excel(xlsx_path: str) -> str:
    if not os.path.exists(xlsx_path): return ""
    try:
        xl = pd.ExcelFile(xlsx_path)
        chunks = []
        for sh in xl.sheet_names:
            df = xl.parse(sh)
            chunks.append(f"[Sheet: {sh}]\n" + df.to_string(index=False))
        return "\n\n".join(chunks)
    except Exception:
        return ""


# --- HTML → plain text ---
# Parses an HTML file and strips non-content tags.
# Notes:
# - Requires BeautifulSoup with the "html5lib" parser for tolerant parsing.
# - Removes <script>, <style>, and <noscript> to avoid boilerplate/noise.
# - Uses separator="\n" to preserve a readable line structure.
def extract_text_from_html(html_path: str) -> str:
    if BeautifulSoup is None:
        raise RuntimeError("beautifulsoup4/html5lib not installed. Run: %pip install beautifulsoup4 html5lib")
    if not os.path.exists(html_path): return ""
    with open(html_path, "r", encoding="utf-8", errors="ignore") as f:
        html = f.read()
    soup = BeautifulSoup(html, "html5lib")
    for bad in soup(["script","style","noscript"]): bad.extract()  # remove non-textual content
    return soup.get_text(separator="\n")


# 1.7) Text Cleaning (Headers/Footers/Page Numbers)

**Goal:** Normalize extracted text to reduce noise before chunking — improves both retrievers and cross-encoders.


In [7]:
# --- Text cleanup utility for extracted documents ---
# Goal:
# - Normalize newlines, remove boilerplate like page numbers and headers,
#   collapse excessive whitespace, and return a clean text block.
def clean_text(text: str) -> str:
    # Fast exit for None/empty strings
    if not text:
        return ""

    # Normalize different newline conventions to '\n'
    t = text.replace("\r\n","\n").replace("\r","\n")

    # --- Line-level filtering: drop empties and page-number artifacts ---
    lines = []
    for ln in t.split("\n"):
        s = ln.strip()
        if not s: 
            continue  # skip blank lines

        # Skip lines that are just digits (likely page numbers like "12")
        if re.fullmatch(r"\d{1,4}", s):
            continue

        # Skip typical "Page X" or "Page X of Y" footers/headers (case-insensitive)
        if re.search(r"\bPage\s+\d+\s*(?:of\s*\d+)?\b", s, flags=re.I):
            continue

        # Keep everything else
        lines.append(ln)

    # Rebuild text from the filtered lines
    t = "\n".join(lines)

    # --- Remove common header patterns like "Company | Form | PageNo" on single lines ---
    # ^...$ with (?m) anchors per line; matches alnum/punct separated by pipes and ends with a small number
    t = re.sub(r"(?m)^\s*[A-Za-z0-9 .,&\-|]{5,}\|\s*\d{1,3}\s*$", "", t)

    # --- Whitespace normalization ---
    # Collapse runs of spaces/tabs inside lines
    t = re.sub(r"[ \t]+", " ", t)
    # Collapse 3+ consecutive newlines to just 2 to keep paragraph breaks
    t = re.sub(r"\n{3,}", "\n\n", t)

    # Final trim of leading/trailing whitespace
    return t.strip()


# 1.8) Build Per-Year Section Files (Ticker × Statement × Year)

**Why sections:** RAG works best with coherent passages. We serialize each (ticker, statement, year) to text.

**Result:** Files like `out/sections_text_yearly/ADP__income_statement__2024.txt`.


In [8]:
# --- Build a readable, per-year text "section" for a given ticker & statement ---
# Inputs:
#   df        : long/tidy financials with columns: ticker, statement, fiscal_year, line_item, value, unit
#   ticker    : e.g., "AAPL"
#   statement : one of {"income_statement","balance_sheet","cash_flow"}
#   year      : fiscal year as int (e.g., 2024)
# Output:
#   Plain-text block summarizing up to 40 largest absolute line items for that (ticker, statement, year),
#   cleaned via clean_text(...); returns "" if no rows found.
def section_blob_year(df: pd.DataFrame, ticker: str, statement: str, year: int) -> str:
    # Slice the subset we care about
    sub = df[(df["ticker"]==ticker) & (df["statement"]==statement) & (df["fiscal_year"]==year)].copy()
    if sub.empty: return ""
    # Title line with pretty-cased statement label
    title = f"{ticker} — {statement.replace('_',' ').title()} — Fiscal Year: {int(year)}"
    lines = [title, "-"*len(title)]
    # Rank by magnitude so the most material items appear first
    sub["abs_val"] = sub["value"].abs()
    # Top-N (40) largest absolute values for readability
    for _, r in sub.sort_values("abs_val", ascending=False).head(40).iterrows():
        # Use no-decimal formatting for compactness, preserving the unit (e.g., USD)
        lines.append(f" - {r['line_item']}: {r['value']:.0f} {r['unit']}")
    # Join and sanitize using the shared text cleaner
    return clean_text("\n".join(lines))


# --- Prepare an output folder to store one text file per (ticker, statement, year) ---
SECTIONS_DIR = "out/sections_text_yearly"
os.makedirs(SECTIONS_DIR, exist_ok=True)

# Track written files for a quick summary & sample preview
written = []

# Iterate deterministically: by ticker, then by statement, then by year
for t in sorted(df_fin["ticker"].unique()):
    years = sorted(df_fin[df_fin["ticker"]==t]["fiscal_year"].unique())
    for stmt in ["income_statement","balance_sheet","cash_flow"]:
        for y in years:
            # Build the section text; skip if nothing to write
            txt = section_blob_year(df_fin, t, stmt, int(y))
            if not txt:
                continue
            # File naming convention: TICKER__statement__YYYY.txt
            fp = f"{SECTIONS_DIR}/{t}__{stmt}__{int(y)}.txt"
            with open(fp, "w", encoding="utf-8") as f:
                f.write(txt)
            written.append(fp)

# Final status message + show a small sample of paths
print(f"[ok] wrote {len(written)} section files → {SECTIONS_DIR}")
written[:6]


[ok] wrote 24 section files → out/sections_text_yearly


['out/sections_text_yearly/AAGH__income_statement__2023.txt',
 'out/sections_text_yearly/AAGH__income_statement__2024.txt',
 'out/sections_text_yearly/AAGH__balance_sheet__2023.txt',
 'out/sections_text_yearly/AAGH__balance_sheet__2024.txt',
 'out/sections_text_yearly/AAGH__cash_flow__2023.txt',
 'out/sections_text_yearly/AAGH__cash_flow__2024.txt']

# 1.9) Generate ≥ 50 Q/A Pairs (Deterministic, Year-Specific)

**What:** Build factual numeric Q/A from the tidy data (used for RAG evaluation and FT training).

**Why:** Ground-truth pairs let us check retrieval quality and numeric accuracy.


In [9]:
# --- Humanize raw line-item names ---
# - Trims whitespace
# - Replaces underscores with spaces
# - Inserts spaces before inner capital letters (CamelCase → spaced)
# - Collapses multiple spaces to one
# - Title-cases the result (e.g., "gross profit" → "Gross Profit")
def pretty_item(name: str) -> str:
    name = name.strip().replace("_"," ")
    name = re.sub(r"(?<=.)([A-Z])", r" \1", name)  # split CamelCase
    name = re.sub(r"\s+", " ", name)
    return name.title()


# --- Format numeric values as compact money strings ---
# Uses short-scale suffixes with two decimals where appropriate.
# Examples:
#   123 → "$123"
#   12_300 → "$12 thousand"
#   5_400_000 → "$5.40 million"
#   7_200_000_000 → "$7.20 billion"
#   1_500_000_000_000 → "$1.50 trillion"
def fmt_money(val: float) -> str:
    v = float(val); a = abs(v)
    if a >= 1e12: return f"${v/1e12:.2f} trillion"
    if a >= 1e9:  return f"${v/1e9:.2f} billion"
    if a >= 1e6:  return f"${v/1e6:.2f} million"
    if a >= 1e3:  return f"${v/1e3:.0f} thousand"
    return f"${v:.0f}"


# --- Items we prefer to ask about first (if present in the data) ---
# Order matters: earlier items get priority when building Q/A.
PREFERRED_ITEMS = [
    "Revenue", "Total Revenue", "Net Income", "Gross Profit",
    "Operating Income", "Ebit", "Ebitda",
    "Total Assets", "Total Liabilities", "Shareholders Equity",
    "Cash And Cash Equivalents", "Free Cash Flow", "Operating Cash Flow"
]


# --- Build natural-language Q/A pairs from a tidy financials DataFrame ---
# Strategy:
#   1) For each (ticker, year), generate questions for PREFERRED_ITEMS found.
#   2) Then, for each statement, add up to 12 top-|value| line items for coverage.
#   3) De-duplicate by question text.
#   4) If we still have fewer than `min_pairs`, top up by global |value| ranking.
# Assumes df columns: ticker | fiscal_year | statement | line_item | value | unit
def build_qa_pairs(df: pd.DataFrame, min_pairs: int = 50) -> List[Dict]:
    pairs = []
    for t in sorted(df["ticker"].unique()):
        for year in sorted(df[df["ticker"]==t]["fiscal_year"].unique()):
            sub = df[(df["ticker"]==t) & (df["fiscal_year"]==year)].copy()
            if sub.empty: 
                continue

            # 1) Preferred items first (if present)
            for pref in PREFERRED_ITEMS:
                # Case-insensitive exact-substring match on the raw line_item text
                hits = sub[sub["line_item"].str.contains(pref, case=False, regex=False)]
                for _, r in hits.iterrows():
                    item = pretty_item(r["line_item"])   # normalize display name
                    val  = fmt_money(r["value"])         # human-friendly money format
                    q = f"What was {t}'s {item} in {int(year)}?"
                    a = f"{t}'s {item} in {int(year)} was {val} ({r['unit']})."
                    pairs.append({"ticker": t, "year": int(year), "statement": r["statement"],
                                  "line_item": item, "question": q, "answer": a})

            # 2) Fill with top-N by absolute value per statement
            for stmt in ["income_statement","balance_sheet","cash_flow"]:
                grp = sub[sub["statement"]==stmt].copy()
                if grp.empty: 
                    continue
                grp["abs_val"] = grp["value"].abs()
                # Take the 12 most material items to keep Q/A concise yet representative
                for _, r in grp.sort_values("abs_val", ascending=False).head(12).iterrows():
                    item = pretty_item(r["line_item"])
                    val  = fmt_money(r["value"])
                    q = f"In {int(year)}, what was {t}'s {item} ({stmt.replace('_',' ').title()})?"
                    a = f"In {int(year)}, {t}'s {item} was {val} ({r['unit']})."
                    pairs.append({"ticker": t, "year": int(year), "statement": stmt,
                                  "line_item": item, "question": q, "answer": a})

    # De-duplicate by question text
    seen, uniq = set(), []
    for p in pairs:
        if p["question"] not in seen:
            seen.add(p["question"])
            uniq.append(p)

    # Ensure minimum count by backfilling from highest-|value| rows globally
    if len(uniq) < min_pairs:
        g = df.copy()
        g["abs_val"] = g["value"].abs()
        for _, r in g.sort_values("abs_val", ascending=False).iterrows():
            q = f"What was {r['ticker']}'s {pretty_item(str(r['line_item']))} in {int(r['fiscal_year'])}?"
            if q in seen: continue
            a = f"{r['ticker']}'s {pretty_item(str(r['line_item']))} in {int(r['fiscal_year'])} was {fmt_money(r['value'])} ({r['unit']})."
            uniq.append({"ticker": r["ticker"], "year": int(r["fiscal_year"]), "statement": r["statement"],
                         "line_item": pretty_item(str(r["line_item"])), "question": q, "answer": a})
            seen.add(q)
            if len(uniq) >= min_pairs:
                break

    return uniq


# --- Materialize Q/A and preview a sample ---
qa_pairs = build_qa_pairs(df_fin, min_pairs=50)
print(f"[ok] built {len(qa_pairs)} Q/A pairs (min 50)")
print("Q:", qa_pairs[0]["question"])
print("A:", qa_pairs[0]["answer"])


[ok] built 376 Q/A pairs (min 50)
Q: What was AAGH's Cost Of Revenue in 2023?
A: AAGH's Cost Of Revenue in 2023 was $-36 thousand (USD).


# 1.10) Validate & Save Q/A + Manifest

**What:** Simple value-extraction sanity check + persist artifacts.

**Why:** Guarantees we have reproducible inputs for RAG and Fine-Tuning (FT).


In [10]:
# --- Extract the first money-like token from an answer string ---
# Matches:
#   - A leading '$'
#   - Digits with optional commas/periods
#   - Optional scale word: thousand | million | billion | trillion (case-insensitive)
# Returns the matched string (e.g., "$1.23 billion") or "" if none found.
def value_in_answer(answer: str) -> str:
    m = re.search(r"\$[\d,.]+(?:\s*(?:thousand|million|billion|trillion))?", answer, flags=re.I)
    return m.group(0) if m else ""

# --- Lightweight sanity-check on generated Q/A pairs ---
# Randomly samples up to `samples` items from `qa` and verifies that the answer text
# contains a money-like token (per value_in_answer). Prints each sampled Q/A and a summary.
def validate_pairs(qa: List[Dict], df: pd.DataFrame, samples: int = 8):
    import random
    ok = 0
    samp = random.sample(qa, k=min(samples, len(qa)))  # safe if qa has fewer than `samples` entries
    for p in samp:
        money = value_in_answer(p["answer"])           # extract "$... [scale]" if present
        ok += int(bool(money))                         # count 1 if token found, else 0
        print("Q:", p["question"])
        print("A:", p["answer"])
        print("✓ money found:", money, "\n")
    print(f"[check] money-like value present in {ok}/{len(samp)} sampled answers.")

# Run the quick validation on the built QA pairs
validate_pairs(qa_pairs, df_fin, samples=8)

# --- Persist artifacts to disk (JSONL + CSV) ---
os.makedirs("out", exist_ok=True)

# Save Q/A pairs as JSON Lines (one JSON object per line) for easy streaming/ingest.
with open("out/qa_pairs.jsonl", "w", encoding="utf-8") as f:
    for p in qa_pairs:
        f.write(json.dumps(p, ensure_ascii=False) + "\n")

# Also save a tabular view for quick inspection in spreadsheets.
pd.DataFrame(qa_pairs).to_csv("out/qa_pairs.csv", index=False)

# --- Build a manifest with dataset metadata and file pointers ---
manifest = {
    "years_required": sorted(YEARS_REQUIRED),
    "selected_tickers": sorted(df_fin["ticker"].unique().tolist()),
    "num_rows_financials": len(df_fin),
    "num_sections_written": len([x for x in os.listdir("out/sections_text_yearly") if x.endswith(".txt")]),
    "num_qa_pairs": len(qa_pairs),
    "files": [
        "out/financials_last2y.csv",
        "out/sections_text_yearly/<ticker>__<statement>__<year>.txt",
        "out/qa_pairs.jsonl",
        "out/qa_pairs.csv"
    ]
}

# Write the manifest as pretty-printed JSON for reproducibility and downstream tooling.
with open("out/dataset_manifest.json", "w", encoding="utf-8") as f:
    json.dump(manifest, f, indent=2)

# --- Final console summary of what was written ---
print("[ok] wrote:")
for p in manifest["files"]:
    print(" -", p)


Q: In 2024, what was AFRM's Total Noncurrent Assets (Balance Sheet)?
A: In 2024, AFRM's Total Noncurrent Assets was $7.05 billion (USD).
✓ money found: $7.05 billion 

Q: In 2023, what was AAGH's Shares ( Diluted) (Cash Flow)?
A: In 2023, AAGH's Shares ( Diluted) was $21.09 billion (USD).
✓ money found: $21.09 billion 

Q: In 2023, what was AAGH's Shares ( Diluted) (Balance Sheet)?
A: In 2023, AAGH's Shares ( Diluted) was $21.09 billion (USD).
✓ money found: $21.09 billion 

Q: What was AAGH's Non- Operating Income ( Loss) in 2024?
A: AAGH's Non- Operating Income ( Loss) in 2024 was $-444 thousand (USD).
✓ money found:  

Q: In 2024, what was AFRM's Net Income (Income Statement)?
A: In 2024, AFRM's Net Income was $-517.76 million (USD).
✓ money found:  

Q: What was AFRM's Non- Operating Income ( Loss) in 2023?
A: AFRM's Non- Operating Income ( Loss) in 2023 was $211.62 million (USD).
✓ money found: $211.62 million 

Q: In 2023, what was AFRM's Depreciation & Amortization (Cash Flow)?


# Device Selection & Config

**What:** Pick CUDA/MPS/CPU and define model names.

**Why:** Consistent, fast inference in RAG and CE (cross-encoder).


In [11]:
# Pick the best available device once
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("[info] CUDA:", torch.cuda.get_device_name(0))
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("[info] Apple MPS")
else:
    DEVICE = torch.device("cpu")
    print("[info] CPU")

# Paths & models
SECTIONS_DIR = "out/sections_text_yearly"  # from Step-1
EMB_MODEL    = "sentence-transformers/all-MiniLM-L6-v2"      # or: "intfloat/e5-small-v2"
RERANKER     = "cross-encoder/ms-marco-MiniLM-L-6-v2"        # fast, ~80MB


[info] CPU


# 2. Retrieval-Augmented Generation (RAG) System Implementation

# 2.1) Load Section Texts & Token-Aware Chunking (100 / 400 tokens)

**RAG detail:** Chunking impacts recall and CE reranking quality. We use token-aware windows with small overlaps to keep semantic cohesion while fitting model limits.


In [12]:
# Suppress only THIS specific FutureWarning message from transformers
warnings.filterwarnings(
    "ignore",
    message=r".*`clean_up_tokenization_spaces` was not set.*",
    category=FutureWarning,
    module=r"transformers\.tokenization_utils_base"
)

# (Optional) also quiet all transformers advisory logs
from transformers.utils import logging as hf_logging
hf_logging.set_verbosity_error()


# After you create tokenizers, set their default behavior explicitly.

# embedding tokenizer (tok)
try:
    tok.clean_up_tokenization_spaces = False
except NameError:
    pass

# cross-encoder tokenizer (ce_tok), if you use one
try:
    ce_tok.clean_up_tokenization_spaces = False
except NameError:
    pass


#Redefine chunk_by_tokens to use a stable decode that avoids the FutureWarning
from transformers import AutoTokenizer
import re

# --- Load all per-year section text files ---
def load_sections(dirpath: str) -> List[Dict]:
    docs = []
    for fp in sorted(glob.glob(os.path.join(dirpath, "*.txt"))):
        base = os.path.basename(fp)  # e.g., AAPL__income_statement__2024.txt
        if base.count("__") != 2:
            continue
        ticker, statement, tail = base.split("__")
        year = int(tail.rsplit(".", 1)[0])
        with open(fp, "r", encoding="utf-8", errors="ignore") as f:
            text = f.read()
        docs.append({"ticker": ticker, "statement": statement, "year": year, "path": fp, "text": text})
    return docs

sections = load_sections(SECTIONS_DIR)
assert sections, f"No section files found in {SECTIONS_DIR}. Run Step-1 first."
print(f"[info] loaded sections: {len(sections)}")

# --- Tokenizer for token-aware chunking ---
tok = AutoTokenizer.from_pretrained(EMB_MODEL)


def _decode(ids):
    # Stable decoding: don't auto "clean up" spaces; skip specials
    return tok.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

def num_tokens(text: str) -> int:
    return len(tok.encode(text, add_special_tokens=False))

def chunk_by_tokens(text: str, max_tokens: int, overlap_tokens: int = 20):
    """
    Token-aware chunking with light sentence alignment and overlap.
    This version suppresses the transformers FutureWarning by controlling decode().
    """
    ids = tok.encode(text, add_special_tokens=False)
    if len(ids) <= max_tokens:
        return [text.strip()] if text.strip() else []

    # light sentence split
    sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z(])', text.strip()) or [text.strip()]

    chunks, cur, cur_len = [], [], 0
    for s in sentences:
        s_ids = tok.encode(s, add_special_tokens=False)
        s_len = len(s_ids)

        # hard-wrap very long sentences
        if s_len > max_tokens:
            start = 0
            while start < s_len:
                end = min(s_len, start + max_tokens)
                piece = _decode(s_ids[start:end]).strip()
                if piece:
                    chunks.append(piece)
                start = max(end - overlap_tokens, end)
            continue

        # pack into current window
        if cur_len + s_len <= max_tokens:
            cur.append(s); cur_len += s_len
        else:
            if cur:
                chunks.append(" ".join(cur).strip())
            # add overlap tail from previous chunk
            if overlap_tokens > 0 and chunks:
                prev_ids = tok.encode(chunks[-1], add_special_tokens=False)
                tail     = _decode(prev_ids[-overlap_tokens:])
                cur, cur_len = [tail, s], len(prev_ids[-overlap_tokens:]) + s_len
            else:
                cur, cur_len = [s], s_len

    if cur:
        chunks.append(" ".join(cur).strip())

    # enforce limit with small tolerance
    return [c for c in chunks if c and num_tokens(c) <= max_tokens + 5]

from hashlib import blake2b
def stable_id(*parts) -> str:
    h = blake2b(digest_size=8)
    for p in parts:
        h.update(str(p).encode("utf-8", "ignore")); h.update(b"|")
    return h.hexdigest()

def build_chunks(sections: List[Dict], sizes=(100, 400), overlap=20) -> List[Dict]:
    out = []
    for doc in sections:
        for max_toks in sizes:
            pieces = chunk_by_tokens(doc["text"], max_tokens=max_toks, overlap_tokens=overlap)
            for j, txt in enumerate(pieces):
                cid = stable_id(doc["ticker"], doc["statement"], doc["year"], max_toks, j)
                out.append({
                    "id": cid,
                    "text": txt,
                    "ticker": doc["ticker"],
                    "statement": doc["statement"],
                    "year": int(doc["year"]),
                    "source": doc["path"],
                    "chunk_size": max_toks,
                    "chunk_index": j,
                    "n_tokens": num_tokens(txt),
                })
    return out

# Rebuild chunks if you want to re-run 2.1 with this patched function:
chunks = build_chunks(sections, sizes=(100, 400), overlap=20)
print("[info] total chunks:", len(chunks))




[info] loaded sections: 24
[info] total chunks: 99


# 2.2) Embedding & Indexing (Dense FAISS + Sparse TF-IDF)

**RAG detail:** Hybrid retrieval fuses dense semantic scores with sparse lexical matches for robust recall.


In [13]:
# --- Sentence embeddings ---
embedder = SentenceTransformer(EMB_MODEL)
print("[info] embedding model:", EMB_MODEL)

texts = [c["text"] for c in chunks]
X = embedder.encode(texts, batch_size=64, show_progress_bar=True, normalize_embeddings=True)
X = np.asarray(X, dtype="float32")
print("[info] embeddings:", X.shape)

# --- Dense FAISS (Inner Product; embeddings are normalized so IP≈cosine) ---
faiss_index = faiss.IndexFlatIP(X.shape[1])
faiss_index.add(X)
print("[info] FAISS ntotal:", faiss_index.ntotal)

# --- Sparse TF-IDF ---
tfidf = TfidfVectorizer(strip_accents="unicode", lowercase=True, max_df=0.95, min_df=1,sublinear_tf=True, ngram_range=(1,2))
Xsp = tfidf.fit_transform(texts)
print("[info] TF-IDF matrix:", Xsp.shape)

# Keep aligned metadata list (same order as 'texts')
meta = chunks


[info] embedding model: sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

[info] embeddings: (99, 384)
[info] FAISS ntotal: 99
[info] TF-IDF matrix: (99, 1296)


# 2.3) Hybrid Retrieval Pipeline (dense + sparse + weighted fusion)

**RAG detail:** We normalize each retriever’s scores and fuse them (`alpha` weights) before CE re-ranking. This balances recall (sparse) and semantic precision (dense).


In [14]:
# Preprocess, dense & sparse search, score fusion
# Purpose:
# - Provide a lightweight retrieval layer that combines dense (vector) and sparse (TF-IDF) signals.
# - Normalize and fuse scores, then return a larger candidate pool for a subsequent cross-encoder reranker.
# Assumptions:
# - Globals defined elsewhere: EMB_MODEL (str), embedder (SentenceTransformer), faiss_index (FAISS),
#   tfidf (TfidfVectorizer), Xsp (sparse TF-IDF matrix), meta (list/dict of doc metadata).


# --- knobs (put near your configs) ---
# Size of fused candidate pool (before cross-encoder). Try 36 or 48 for better recall.
FUSE_RETURN = 36          # how many fused candidates you keep before CE (try 36 or 48)
# Overfetch factor for each retriever relative to FUSE_RETURN to allow better fusion diversity.
FUSE_FETCH_MULT = 2       # how much to fetch from each retriever relative to FUSE_RETURN


# Light stopwords (kept tiny on purpose)
# Rationale: keep query semantics intact while trimming very common function words.
STOPWORDS = set("""
a an and are as at be but by for if in into is it its of on or such that the their then there these this to with
""".split())

# --- Basic query preprocessing ---
# Lowercase, strip, keep alnum & a few financial symbols, remove tiny stopword list.
def preprocess_query(q: str) -> str:
    q = (q or "").lower().strip()
    q = re.sub(r"[^a-z0-9 %$.,/-]+", " ", q)
    q = " ".join(w for w in q.split() if w not in STOPWORDS)
    return q

# E5 models expect "query: " prefix. MiniLM does not.
# This helper auto-adds the prefix based on EMB_MODEL naming.
def _maybe_prefix_query(q: str) -> str:
    return f"query: {q}" if "e5" in EMB_MODEL.lower() else q

# --- Dense retrieval (FAISS over normalized embeddings) ---
# Returns: List of (doc_idx, similarity) pairs.
def dense_search(query: str, k: int = 30) -> List[Tuple[int, float]]:
    qn = _maybe_prefix_query(preprocess_query(query))
    qv = embedder.encode([qn], normalize_embeddings=True)
    sims, idxs = faiss_index.search(np.asarray(qv, dtype="float32"), k)
    return list(zip(idxs[0].tolist(), sims[0].tolist()))

# --- Sparse retrieval (TF-IDF + cosine similarity) ---
# Returns: List of (doc_idx, similarity) pairs, highest first.
def sparse_search(query: str, k: int = 30) -> List[Tuple[int, float]]:
    qn = preprocess_query(query)
    qv = tfidf.transform([qn])
    sims = cosine_similarity(qv, Xsp)[0]
    top = sims.argsort()[::-1][:k]
    return [(int(i), float(sims[i])) for i in top]

# --- Min-max normalize a score dict to [0,1] ---
# Robustness: if all scores equal (hi≈lo), return zeros to avoid NaNs/inf.
def _minmax(scores: Dict[int, float]) -> Dict[int, float]:
    if not scores: return {}
    vals = np.array(list(scores.values()), dtype="float32")
    lo, hi = float(vals.min()), float(vals.max())
    if hi - lo < 1e-9:
        return {i: 0.0 for i in scores}
    return {i: (s - lo) / (hi - lo) for i, s in scores.items()}

# --- Hybrid retrieval with score fusion ---
# alpha controls dense vs sparse contribution after min-max (alpha=1 → dense only, 0 → sparse only).
# k_ctx is the downstream context size (not used here; fusion returns a larger pool for CE).
def hybrid_retrieve(query: str, k_ctx: int = 12, alpha: float = 0.6) -> List[Dict]:
    """
    alpha ∈ [0,1]: weight for dense score after min-max normalization.
    Returns FUSE_RETURN fused results (bigger pool for CE).
    """
    fetch_k = max(FUSE_RETURN, int(FUSE_FETCH_MULT * FUSE_RETURN))

    # fetch more from each side
    d = dense_search(query, k=fetch_k)
    s = sparse_search(query, k=fetch_k)

    # Build raw score dicts, then normalize independently per modality
    d_scores = {i: sc for i, sc in d}
    s_scores = {i: sc for i, sc in s}
    d_n, s_n = _minmax(d_scores), _minmax(s_scores)

    # Union doc ids across modalities, fuse with weighted sum
    all_ids = set(d_scores) | set(s_scores)
    fused = []
    for i in all_ids:
        fd, fs = d_n.get(i, 0.0), s_n.get(i, 0.0)
        score = alpha*fd + (1-alpha)*fs
        m = dict(meta[i])                 # copy metadata for this doc id
        m["dense_score"] = float(fd)      # keep normalized modality scores for debugging
        m["sparse_score"] = float(fs)
        m["fused_score"]  = float(score)  # final fused score
        fused.append(m)

    # Sort by fused score (desc) and return a larger candidate pool for cross-encoder rerank
    fused.sort(key=lambda x: x["fused_score"], reverse=True)
    return fused[:FUSE_RETURN]   # <-- bigger pool before CE


# 2.4) Advanced RAG — Re-Ranking with a Cross-Encoder (CE)

**What:** A **cross-encoder** scores *(query, passage)* pairs jointly (full attention), providing strong lexical/semantic relevance.

**Why it matters (extra detail):**
- Bi-encoders (embeddings) are fast but approximate.  
- CE adds a final precision layer → better top-k contexts → more accurate answers.  
- We use a CE (`ms-marco-MiniLM-L-6-v2`) to keep latency low in Colab.


In [15]:
# Load a cross-encoder and use it to re-rank the fused candidates
# Purpose:
# - Take a small candidate set from hybrid retrieval and sort it by semantic relevance to the query.
# - Uses a pretrained cross-encoder that jointly encodes (query, passage) pairs and outputs a relevance score.
# Assumptions:
# - Global `DEVICE` is defined (e.g., "cuda" or "cpu").
# - Each element of `candidates` contains at least a "text" field (the passage/content to score).
# - `transformers` and `torch` are available.

RERANKER = "cross-encoder/ms-marco-MiniLM-L-6-v2"  # fast & tiny, good trade-off of speed vs quality
ce_tok   = AutoTokenizer.from_pretrained(RERANKER)  # tokenizer for paired (query, text) inputs
ce_model = AutoModelForSequenceClassification.from_pretrained(RERANKER).to(DEVICE)  # moves model to DEVICE
ce_model.eval()  # inference mode (disables dropout etc.)

@torch.no_grad()  # disable gradient computation for faster, memory-efficient inference
def rerank_with_cross_encoder(query: str, candidates: List[Dict], topk: int = 6, return_probs: bool = True) -> List[Dict]:
    # Quick exit if there are no candidates to score
    if not candidates:
        return []

    # Build (query, passage) pairs expected by cross-encoders
    pairs = [(query, c["text"]) for c in candidates]

    # Tokenize as sentence pairs with truncation/padding up to the model's max_length
    batch = ce_tok.batch_encode_plus(
        pairs, padding=True, truncation=True, max_length=512, return_tensors="pt"
    ).to(DEVICE)

    # Forward pass → raw logits
    logits = ce_model(**batch).logits

    # Handle both possible head configurations:
    # - Regression head: shape (..., 1) → higher is "more relevant"
    # - Classification head: shape (..., 2) → use the positive-class (index 1) logit
    if logits.shape[-1] == 1:
        raw = logits.squeeze(-1)                      # shape: [N]
        probs = torch.sigmoid(raw) if return_probs else None  # convert to [0,1] if requested
        sort_key = raw                                # sort by raw score
    else:
        raw = logits[:, 1]                            # take positive-class logit (relevant)
        probs = torch.softmax(logits, dim=-1)[:, 1] if return_probs else None
        sort_key = raw

    # Sort indices by descending relevance
    order = torch.argsort(sort_key, descending=True).tolist()

    # Materialize topk items, attaching scores (and probabilities if requested)
    ranked = []
    for i in order[:topk]:
        item = dict(candidates[i])                    # shallow copy to avoid mutating input
        item["rerank_logit"] = float(raw[i])          # raw model score for transparency/debugging
        if return_probs:
            item["rerank_prob"] = float(probs[i])     # calibrated-ish probability (depends on head)
        ranked.append(item)

    return ranked


## **2.4.1:** build ticker↔company maps (from df_fin)

In [16]:
# Build maps once from your tidy dataframe
TICKER_TO_COMPANY = {}
COMPANY_TO_TICKER = {}

if "df_fin" in globals():
    for t, grp in df_fin.groupby("ticker"):
        name = str(grp["company"].iloc[0]).strip()
        if name:
            TICKER_TO_COMPANY[t] = name
            COMPANY_TO_TICKER[name.lower()] = t

# Keep any manual aliases you already added
# NAME_ALIASES should already exist from your earlier alias cell; if not, create a minimal one:
NAME_ALIASES = NAME_ALIASES if "NAME_ALIASES" in globals() else {
    "automatic data processing": "ADP",
    "adp": "ADP",
    "affirm": "AFRM",
    "affirm holdings": "AFRM",
    "aehr": "AEHR",
    "aehr test systems": "AEHR",
    "america great health": "AAGH",
    "aagh": "AAGH",
}


## 2.4.2 canonicalize query entities (alias → ticker [+ company])

In [17]:

def canonicalize_query_entities(q: str) -> str:
    """
    Replace alias/company mentions with the canonical 'TICKER CompanyName' prefix,
    then append the original task (e.g., 'revenue 2024').
    """
    # Defensive: accept None/empty and coerce to string
    q_raw = q or ""
    # Lowercased copy for case-insensitive alias detection, keep q_raw intact for fallback
    q_low = q_raw.lower()

    # Initialize chosen ticker (if any)
    ticker = None
    # If a global parse_query(...) helper exists, use it to extract ticker/year
    if "parse_query" in globals():
        t, y = parse_query(q_raw)
        ticker = t

    # If we still don't have a ticker, try a lightweight alias→ticker pass
    if not ticker:
        # NAME_ALIASES is expected as {alias_string: "TICKER"}
        for alias, tk in NAME_ALIASES.items():
            # Heuristic: short aliases matched as whole words; long names via substring
            if (len(alias) <= 6 and re.search(rf"\b{re.escape(alias)}\b", q_low)) or (len(alias) > 6 and alias in q_low):
                ticker = tk
                break

    # If we found a ticker, prepend its canonical "TICKER Company" prefix
    if ticker:
        # Map ticker→Company Name; blank if unknown
        company = TICKER_TO_COMPANY.get(ticker, "")
        prefix = f"{ticker} {company}".strip()
        # Remove any alias strings that pointed to this ticker to avoid duplication in final text
        for alias, tk in NAME_ALIASES.items():
            if tk == ticker:
                q_low = re.sub(rf"\b{re.escape(alias)}\b", "", q_low)
        # Normalize whitespace after removals
        q_low = re.sub(r"\s+", " ", q_low).strip()
        # Compose the canonicalized query: "<TICKER> <Company> <rest>"
        canonical = f"{prefix} {q_low}".strip()
        return canonical

    # Fallback: return the original query unchanged if no ticker was inferred
    return q_raw


## 2.4.3 augment passages for re-ranking (prepend ticker + company)

In [18]:
# --- Augment a passage for cross-encoder reranking ---
# Purpose:
# - Increase lexical overlap between alias-based queries and passages by explicitly
#   prepending the canonical "<TICKER> <CompanyName>" line to the passage text.
# Inputs:
# - p: dict expected to contain:
#       - "ticker": stock ticker string (e.g., "AAPL")
#       - "text"  : the original passage/content
# Requirements:
# - Global mapping TICKER_TO_COMPANY: { "TICKER": "Company Name", ... }
# Output:
# - Single string composed as:
#       "<TICKER> <CompanyName>\n<original passage>"
def augment_passage_for_rerank(p: dict) -> str:
    """
    Prepend 'TICKER CompanyName' to the passage text seen by the cross-encoder
    to increase lexical overlap with alias queries.
    """
    ticker = p.get("ticker", "")                      # pull ticker if available; else empty
    company = TICKER_TO_COMPANY.get(ticker, "")       # map ticker → company name (may be "")
    header = f"{ticker} {company}".strip()            # build "TICKER CompanyName" (trim if one part missing)
    text = p.get("text", "")                          # original passage text (default to empty)
    return (header + "\n" + text).strip()             # join with newline; strip to avoid leading/trailing blanks


## 2.4.4 use canonical query + augmented passage in re-ranker

In [19]:
# Purpose:
# - Ensure a cross-encoder reranker is available.
# - When called, canonicalize the query (e.g., add "TICKER Company") and augment each passage
#   to improve lexical overlap for alias-style queries before scoring.

# (Re)load once if not present
# Lazy-initialize tokenizer/model only if they don't already exist in the global scope.
if "ce_model" not in globals():
    RERANKER = "cross-encoder/ms-marco-MiniLM-L-6-v2"  # compact, fast cross-encoder
    ce_tok   = AutoTokenizer.from_pretrained(RERANKER)  # pairwise tokenizer (query, passage)
    ce_model = AutoModelForSequenceClassification.from_pretrained(RERANKER).to(DEVICE)  # move to CPU/GPU
    ce_model.eval()  # inference mode

@torch.no_grad()  # no gradients needed for inference (faster, less memory)
def rerank_with_cross_encoder(query: str, candidates: list, topk: int = 6, return_probs: bool = True) -> list:
    if not candidates:
        return []  # nothing to score → return early

    # canonicalize query entities (alias -> TICKER Company)
    # Example: "apple revenue 2024" → "AAPL Apple Inc revenue 2024"
    q_can = canonicalize_query_entities(query)

    # augment passages with 'TICKER Company' header
    # Each pair is (canonicalized_query, augmented_passage_text)
    pairs = [(q_can, augment_passage_for_rerank(c)) for c in candidates]

    # Tokenize query–passage pairs with truncation/padding for the model
    batch = ce_tok.batch_encode_plus(
        pairs, padding=True, truncation=True, max_length=512, return_tensors="pt"
    ).to(DEVICE)

    # Forward pass → logits per pair
    logits = ce_model(**batch).logits

    # handle regression vs 2-class heads
    # - Regression head (1-d): higher logit = more relevant; optionally map with sigmoid to [0,1]
    # - Classification head (2-d): use positive-class (index 1) logit/probability
    if logits.shape[-1] == 1:
        raw = logits.squeeze(-1)
        probs = torch.sigmoid(raw) if return_probs else None
        sort_key = raw
    else:
        raw = logits[:, 1]
        probs = torch.softmax(logits, dim=-1)[:, 1] if return_probs else None
        sort_key = raw

    # Sort candidates by descending relevance and keep top-k
    order = torch.argsort(sort_key, descending=True).tolist()
    ranked = []
    for i in order[:topk]:
        item = dict(candidates[i])          # shallow copy to avoid mutating input
        item["rerank_logit"] = float(raw[i])
        if return_probs:
            item["rerank_prob"] = float(probs[i])
        ranked.append(item)

    return ranked


## 2.4.5 Also use canonical query for dense/sparse search

**This helps TF-IDF & embeddings when users type the long company name.**

In [20]:
# Wrap your existing search fns if they exist
# Purpose:
# - Keep references to the *original* dense/sparse search so you can revert/compare if needed.
# - Override both search functions to first canonicalize query entities (alias → "TICKER Company"),
#   then run the usual preprocessing and retrieval. This boosts lexical alignment with documents.
_orig_dense_search  = dense_search
_orig_sparse_search = sparse_search

def dense_search(query: str, k: int = 30):
    # Canonicalize user query: prepend/normalize to "<TICKER> <Company> ..." when detectable.
    q_can = canonicalize_query_entities(query)
    # Preserve your E5 prefixing logic via _maybe_prefix_query; also do light normalization.
    qn = _maybe_prefix_query(preprocess_query(q_can))
    # Encode to an embedding vector (normalized) and search the FAISS index for top-k neighbors.
    qv = embedder.encode([qn], normalize_embeddings=True)
    sims, idxs = faiss_index.search(np.asarray(qv, dtype="float32"), k)
    # Return list of (document_index, similarity) pairs.
    return list(zip(idxs[0].tolist(), sims[0].tolist()))

def sparse_search(query: str, k: int = 30):
    # Same canonicalization path for TF-IDF: improves overlap with doc text/headers.
    q_can = canonicalize_query_entities(query)
    # Transform the preprocessed query into the TF-IDF space.
    qv = tfidf.transform([preprocess_query(q_can)])
    # Compute cosine similarity against the sparse matrix Xsp and take top-k by score.
    sims = cosine_similarity(qv, Xsp)[0]
    top = sims.argsort()[::-1][:k]
    # Return list of (document_index, similarity) pairs (as plain Python types).
    return [(int(i), float(sims[i])) for i in top]


## 2.4.6 Add company aliases → map natural-language names to tickers

In [21]:
# Alias-aware query parser
# 1) Build name→ticker from your dataset (df_fin) if available
COMPANY_NAME_TO_TICKER = {}
if "df_fin" in globals():
    # take first company name per ticker
    for t, grp in df_fin.groupby("ticker"):
        nm = str(grp["company"].iloc[0])
        if nm:
            COMPANY_NAME_TO_TICKER[nm.lower()] = t

# 2) Manual aliases for your four tickers
#    Feel free to add more terms users might type.
NAME_ALIASES = {
    # AAGH
    "america great health": "AAGH",
    "aagh": "AAGH",

    # ADP
    "automatic data processing": "ADP",
    "adp": "ADP",
    "adp inc": "ADP",
    "adp payroll": "ADP",
    "adp workforce now": "ADP",

    # AEHR
    "aehr": "AEHR",
    "aehr test": "AEHR",
    "aehr test systems": "AEHR",

    # AFRM
    "affirm": "AFRM",
    "affirm holdings": "AFRM",
    "affirm inc": "AFRM",
    "buy now pay later affirm": "AFRM",
    "bnpl affirm": "AFRM",
}

# Merge company names from the dataset into aliases (non-destructive)
for nm, tk in COMPANY_NAME_TO_TICKER.items():
    NAME_ALIASES.setdefault(nm, tk)

# Optionally, restrict ticker detection to your curated set
KNOWN_TICKERS = set(['AAGH', 'ADP', 'AEHR', 'AFRM'])

def parse_query(query: str):
    """
    Returns (ticker, year) parsed from a user query.
    Priority: explicit uppercase ticker token -> alias match -> company name substring.
    """
    q_raw = query or ""
    q_low = q_raw.lower()

    # 1) explicit uppercase ticker tokens (AAGH, ADP, AEHR, AFRM)
    tickers_in_text = re.findall(r"\b([A-Z]{2,6})\b", q_raw)
    for t in tickers_in_text:
        if not KNOWN_TICKERS or t in KNOWN_TICKERS:
            ticker = t
            break
    else:
        ticker = None

    # 2) alias matches with word boundaries (e.g., "adp", "affirm")
    if not ticker:
        for alias, tk in NAME_ALIASES.items():
            # Use word boundaries for short aliases; substring for long company names
            if len(alias) <= 6:
                if re.search(rf"\b{re.escape(alias)}\b", q_low):
                    ticker = tk; break
            else:
                if alias in q_low:
                    ticker = tk; break

    # 3) (already covered) but kept as fallback: company name substring map
    if not ticker:
        for nm, tk in COMPANY_NAME_TO_TICKER.items():
            if nm in q_low:
                ticker = tk; break

    # Year (first 20xx in the query)
    m_year = re.search(r"\b(20\d{2})\b", q_raw)
    year = int(m_year.group(1)) if m_year else None

    return ticker, year

# --- quick smoke tests ---
tests = [
    "affirm 2024 revenue",
    "automatic data processing balance sheet 2023",
    "aehr operating cash flow 2024",
    "america great health net income in 2024",
    "ADP revenue 2024",
]
for q in tests:
    print(q, "->", parse_query(q))


affirm 2024 revenue -> ('AFRM', 2024)
automatic data processing balance sheet 2023 -> ('ADP', 2023)
aehr operating cash flow 2024 -> ('AEHR', 2024)
america great health net income in 2024 -> ('AAGH', 2024)
ADP revenue 2024 -> ('ADP', 2024)


## 2.4.7 Make sure parse_query(...) from the alias cell is defined.
**Also make sure you have: hybrid_retrieve(...), rerank_with_cross_encoder(...)**

In [22]:
# Lightweight metric detection + context retrieval with hybrid search and cross-encoder rerank
from typing import Optional, List, Dict

# Map canonical metric names to their originating financial statement.
# Used to bias retrieval toward the relevant statement for the asked metric.
METRIC_TO_STMT = {
    "revenue": "income_statement",
    "gross profit": "income_statement",
    "net income": "income_statement",
    "operating income": "income_statement",
    "total assets": "balance_sheet",
    "total liabilities": "balance_sheet",
    "shareholders equity": "balance_sheet",
    "operating cash flow": "cash_flow",
}

# very small keyword → canonical metric detector
# Heuristic detector:
# - Checks multi-word keys first (order matters), then singles and common synonyms ("sales"→"revenue").
# - Returns a canonical metric string or None if not found.
def detect_metric(query: str) -> Optional[str]:
    q = query.lower()
    # check multiword keys first
    for key in ["operating cash flow", "gross profit", "net income", "operating income",
                "total assets", "total liabilities", "shareholders equity"]:
        if key in q:
            return key
    # singles
    if "revenue" in q or "sales" in q:
        return "revenue"
    return None

# Main retrieval routine:
# 1) Parse query to infer (ticker, year) when possible.
# 2) Detect metric→statement to steer retrieval.
# 3) Pull a larger fused candidate pool via hybrid_retrieve (dense+sparse).
# 4) Filter candidates by (ticker/year/statement) with graceful backoffs.
# 5) Rerank the surviving candidates using a cross-encoder and return top-k contexts.
def retrieve_contexts(query: str, k_ctx: int = 6, alpha: float = 0.6):
    want_ticker, want_year = parse_query(query) if "parse_query" in globals() else (None, None)
    metric = detect_metric(query)
    want_stmt = METRIC_TO_STMT.get(metric)

    # Fetch a larger pool than k_ctx to give the reranker room.
    pool = hybrid_retrieve(query, k_ctx=max(12, k_ctx*2), alpha=alpha)

    # Helper: filter by optional (ticker t, year y, statement s)
    def f(items, t=None, y=None, s=None):
        out = items
        if t is not None: out = [x for x in out if x.get("ticker")==t]
        if y is not None: out = [x for x in out if int(x.get("year",-1))==int(y)]
        if s is not None: out = [x for x in out if x.get("statement")==s]
        return out

    # Progressive backoff filtering to retain enough candidates for reranking
    cand = f(pool, want_ticker, want_year, want_stmt)
    if len(cand) < k_ctx and want_ticker: cand = f(pool, want_ticker, want_year, None)
    if len(cand) < k_ctx and want_year:   cand = f(pool, None, want_year, want_stmt)
    if len(cand) < k_ctx and want_stmt:   cand = f(pool, None, None, want_stmt)
    if len(cand) < k_ctx:                 cand = pool[:max(30, 3*k_ctx)]

    # Final cross-encoder rerank; return top-k with probabilities for transparency
    return rerank_with_cross_encoder(query, cand, topk=k_ctx, return_probs=True)

# Demo queries — replace with your own
# Shows how the pipeline reacts to aliases, tickers, and varied phrasing.
for q in [
    "automatic data processing 2024 revenue",
     "ADP 2024 revenue",
    "affirm total assets 2024",
    "aehr operating cash flow 2023",
    "america great health net income in 2024",
]:
    print("\n[Q]", q)
    ctxs = retrieve_contexts(q, k_ctx=5, alpha=0.6)  # uses your existing 2.3/2.4 functions
    for c in ctxs:
        # Preview top contexts with reranker probability and key metadata
        print(f" -> p={c.get('rerank_prob',0.0):.3f} | {c['ticker']} | {c['statement']} | year={c['year']} | size={c['chunk_size']}")
        print("    ", c["text"][:180].replace("\n"," "), "...")



[Q] automatic data processing 2024 revenue
 -> p=1.000 | ADP | income_statement | year=2024 | size=100
     adp — income statement — fiscal year : 2024 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - revenue : 19202600000 usd - gross profit : 1015250 ...
 -> p=0.999 | ADP | income_statement | year=2024 | size=400
     ADP — Income Statement — Fiscal Year: 2024 ------------------------------------------  - Revenue: 19202600000 USD  - Gross Profit: 10152500000 USD  - Cost of Revenue: -9050100000 U ...
 -> p=0.994 | ADP | cash_flow | year=2024 | size=100
     adp — cash flow — fiscal year : 2024 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - net change in long term investment : - 6835300000 usd - net cash from o ...
 -> p=0.991 | ADP | balance_sheet | year=2024 | size=100
     adp — balance sheet — fiscal year : 2024 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - total liabilities & equity : 5

# 2.5) Response Generation — Deterministic Numeric Lookup + RAG Fallback (with citations)

**Flow (extra RAG detail):**
1) Parse *(ticker, year, metric)* from query.  
2) Try **structured** (exact) lookup from `df_fin` (authoritative).  
3) If not found, use **RAG**: hybrid retrieve → CE re-rank → short summary + citations.


In [23]:
# Minimal, dependency-free answerer:
#  - parse (ticker, year) with your parse_query(...)
#  - map metric aliases ("revenue", "operating cash flow", "total assets", "net income", etc.)
#  - look up the exact value in df_fin
#  - if not found, fall back to top RAG contexts and summarize



# map friendly metric names -> list of df_fin line_item regexes to try (order matters)
METRIC_PATTERNS = {
    "revenue": [
        r"^revenue$", r"^total revenue$", r"net sales", r"sales revenue"
    ],
    "net income": [
        r"^net income$", r"net profit", r"profit attributable", r"earnings$"
    ],
    "gross profit": [
        r"^gross profit$"
    ],
    "operating income": [
        r"^operating income$", r"^operating profit$"
    ],
    "operating cash flow": [
        r"^net cash from operating activities$", r"operating cash flow", r"cash flow from operations"
    ],
    "total assets": [
        r"^total assets$"
    ],
    "total liabilities": [
        r"^total liabilities$"
    ],
    "shareholders equity": [
        r"^shareholders'? equity$", r"^stockholders'? equity$", r"^total equity$"
    ],
}


def citations_from_contexts(ctxs, limit: int = 3):
    seen, cites = set(), []
    for c in ctxs:
        key = (c.get("source"), int(c.get("chunk_index", -1)))
        if key in seen:
            continue
        seen.add(key)
        base = os.path.basename(c["source"])
        cites.append(f"{base}#chunk{c['chunk_index']}")
        if len(cites) >= limit:
            break
    return cites




## 2.5.1 Numeric Unit Normalization (e.g., “thousands” → base USD) + Guarded Lookup

**Why:** Company units vary (“USD (millions)”, “USD (thousands)”). We normalize to **base USD** to compare and evaluate fairly.


In [24]:
# Parse common unit strings and normalize all numbers to base units (USD, shares, etc.).
# Scope:
# - Detects magnitude scales like thousand/million/billion/trillion from free-form unit strings.
# - Extracts a major currency code when present (defaults to USD).
# - Converts values to base units by multiplying the appropriate scale factor.
# Note:
# - No FX conversion is performed here; only magnitude scaling.

import re
from typing import Tuple, Optional

# Patterns for compact scale markers commonly seen in filings/notes.
# Examples matched:
#   "k", "000s" → thousand
#   "m", "mn", "mm" → million
#   "b", "bn" → billion
#   "t", "tn" → trillion
_UNIT_SCALE_PATTERNS = [
    (r"\b(thousand|k|000s)\b", 1e3),
    (r"\b(million|mn|mm|m)\b", 1e6),
    (r"\b(billion|bn|b)\b",    1e9),
    (r"\b(trillion|tn|t)\b",   1e12),
]

def _detect_scale(unit_str: str) -> float:
    # Normalize casing/whitespace for robust matching
    u = (unit_str or "").lower().strip()
    # Handle verbose/parenthetical variants commonly seen in tables, e.g., "USD (thousands)"
    if "thousand" in u or "(thousand" in u: return 1e3
    if "million"  in u or "(million"  in u: return 1e6
    if "billion"  in u or "(billion"  in u: return 1e9
    if "trillion" in u or "(trillion" in u: return 1e12
    # Fallback to compact markers (bn, mn, mm, k, etc.)
    for pat, mul in _UNIT_SCALE_PATTERNS:
        if re.search(pat, u):
            return mul
    # Default: treat as already in base units
    return 1.0

def _detect_currency(unit_str: str) -> str:
    # Extract a currency code when present; default to USD if none detected.
    u = (unit_str or "").upper()
    m = re.search(r"\b(USD|EUR|GBP|JPY|CNY|CAD|AUD)\b", u)
    return m.group(1) if m else "USD"

def normalize_value(value: float, unit_str: str) -> Tuple[float, str]:
    """
    Returns (normalized_value, normalized_unit). Example:
      (19.2, 'USD billion') -> (19_200_000_000, 'USD')
      (4157, 'USD thousands') -> (4_157_000, 'USD')
    """
    mul = _detect_scale(unit_str)   # decide scale multiplier from the unit string
    cur = _detect_currency(unit_str)  # pick currency code if present (else USD)
    return float(value) * mul, cur

# Quick sanity checks
# Demonstrates typical inputs and the corresponding normalized outputs.
tests = [
    (1, "USD (thousands)"),
    (2.5, "usd million"),
    (3, "USD bn"),
    (4.2, "eur billion"),
    (500, "USD"),
]
for v,u in tests:
    nv, nu = normalize_value(v,u)
    print(f"{v} {u}  ->  {nv:.0f} {nu}")


1 USD (thousands)  ->  1000 USD
2.5 usd million  ->  2500000 USD
3 USD bn  ->  3000000000 USD
4.2 eur billion  ->  4200000000 EUR
500 USD  ->  500 USD


## 2.5.2 Patch the lookup to return normalized values

In [25]:
# --- SAFE lookup (no chained assignment) ---
def lookup_metric(df_fin: pd.DataFrame, ticker: str, year: int, metric: str):
    if metric not in METRIC_PATTERNS:
        return None

    sub = df_fin.loc[
        (df_fin["ticker"] == ticker) & (df_fin["fiscal_year"] == year)
    ].copy()
    if sub.empty:
        return None

    for pat in METRIC_PATTERNS[metric]:
        m = sub.loc[sub["line_item"].str.contains(pat, case=False, regex=True)].copy()
        if not m.empty:
            row = (
                m.assign(abs_val=m["value"].abs())   # <- no direct m["abs_val"] write
                 .nlargest(1, "abs_val")
                 .iloc[0]
            )
            norm_val, norm_unit = normalize_value(row["value"], row["unit"])
            return {
                "line_item": row["line_item"],
                "value": float(row["value"]),     # raw
                "unit": row["unit"],              # raw
                "value_norm": norm_val,           # normalized
                "unit_norm": norm_unit,           # normalized unit
                "statement": row["statement"],
            }
    return None

# And update answer_query to use value_norm/unit_norm when present
def answer_query(query: str, k_ctx: int = 5, alpha: float = 0.6) -> Dict:
    ticker, year = parse_query(query) if "parse_query" in globals() else (None, None)
    metric = detect_metric(query)

    direct = None
    if ticker and year:
        if metric:
            direct = lookup_metric(df_fin, ticker, year, metric)
        # NEW: try full line_item match if metric lookup failed
        if direct is None:
            li = extract_line_item_from_question(query, ticker, year, df_fin)
            if li:
                # exact/fuzzy line_item lookup (normalized)
                sub = df_fin.loc[
                    (df_fin["ticker"]==ticker) & (df_fin["fiscal_year"]==year) &
                    (df_fin["line_item"].str.contains(re.escape(li), case=False, regex=True))
                ].copy()
                if not sub.empty:
                    row = (sub.assign(abs_val=sub["value"].abs())
                             .nlargest(1, "abs_val").iloc[0])
                    norm_val, norm_unit = normalize_value(row["value"], row["unit"])
                    direct = {
                        "line_item": row["line_item"],
                        "value_norm": norm_val,
                        "unit_norm": norm_unit,
                        "statement": row["statement"],
                    }

    # always retrieve for citations/fallback
    ctxs = retrieve_contexts(query, k_ctx=k_ctx, alpha=alpha)

    if direct:
        val_str = fmt_money(direct["value_norm"])
        return {
            "answer": f"{ticker} {year} { (metric.title() if metric else direct['line_item']) }: {val_str} ({direct['unit_norm']}).",
            "source": "structured_lookup",
            "ticker": ticker, "year": year,
            "metric": (metric or direct["line_item"]),
            "value_norm": direct["value_norm"], "unit_norm": direct["unit_norm"],
            "statement": direct["statement"],
            "citations": citations_from_contexts(ctxs)
        }

    # fallback summary
    bullets = []
    for c in ctxs[:min(3, len(ctxs))]:
        txt = c["text"].strip().split("\n")[0]
        bullets.append(f"• {txt[:220]}{'…' if len(txt)>220 else ''}")
    return {
        "answer": "RAG summary:\n" + "\n".join(bullets) if bullets else "RAG: no contexts.",
        "source": "rag_fallback",
        "ticker": ticker, "year": year, "metric": metric,
        "citations": citations_from_contexts(ctxs)
    }


## 2.5.3 Improving numerical accuracy

In [26]:
def _normalize(s: str) -> str:
    s = (s or "").lower()
    s = re.sub(r"[^a-z0-9 &/\-]+", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def _token_set(s: str) -> set:
    toks = [t for t in _normalize(s).split() if t not in {"the","a","an","of","and","to","in"}]
    return set(toks)

def extract_line_item_from_question(q: str, ticker: str, year: int, df: pd.DataFrame) -> str | None:
    """
    Try to pull the line-item phrase from the question and map it to the closest df_fin line_item.
    Handles both:
      - "What was TICKER's <item> in 2024?"
      - "In 2024, what was TICKER's <item> (Balance Sheet)?"
    """
    qn = _normalize(q)
    # strip ticker & year words to leave the item phrase behind
    qn = qn.replace(_normalize(ticker), " ")
    qn = re.sub(r"\b20\d{2}\b", " ", qn)
    qn = re.sub(r"\b(?:what|was|were|company|s)\b", " ", qn)
    qn = re.sub(r"\s+", " ", qn).strip()

    # candidate item tokens from question
    qset = _token_set(qn)
    if not qset:
        return None

    # compare with all line_items for that ticker+year (token Jaccard overlap)
    sub = df.loc[(df["ticker"]==ticker) & (df["fiscal_year"]==year), ["line_item"]].drop_duplicates()
    best, best_score = None, 0.0
    for li in sub["line_item"]:
        lset = _token_set(li)
        if not lset:
            continue
        inter = len(qset & lset)
        union = len(qset | lset)
        score = inter / union if union else 0.0
        if score > best_score:
            best, best_score = li, score
    return best if best_score >= 0.3 else None  # threshold can be tuned


# 2.5.4 Eval on the Q/A Set — Accuracy@1 & Retrieval Recall@k

**What:** Measure numeric accuracy and whether top-k contexts include the right ticker/year/statement.

**Why:** Gives quick signal on RAG quality before any FT.


In [27]:
# Evaluate:
#  • Accuracy@1: compare predicted numeric vs ground-truth (normalized) within tolerance
#  • Retrieval Recall@k: whether the correct (ticker, year, statement) is present in top-k contexts
#
# Notes:
# - Expects prior steps to have produced `out/qa_pairs.jsonl` and `out/financials_last2y.csv`.
# - Uses your pipeline’s `answer_query`, `retrieve_contexts`, and `normalize_value` defined elsewhere.
# - Regex (`re`) is used to parse "$X [thousand/million/billion/trillion]" from free-form answers.

# Load your Q/A pairs (created in Step-1). If you kept them in memory, you can skip the file load.
QA_PATH = "out/qa_pairs.jsonl"
qa_pairs_eval = []
with open(QA_PATH, "r", encoding="utf-8") as f:
    for line in f:
        qa_pairs_eval.append(json.loads(line))

# Money pretty-printer used for any optional logging/inspection (not strictly required for metrics).
def fmt_money(val: float) -> str:
    v = float(val); a = abs(v)
    if a >= 1e12: return f"${v/1e12:.2f} trillion"
    if a >= 1e9:  return f"${v/1e9:.2f} billion"
    if a >= 1e6:  return f"${v/1e6:.2f} million"
    if a >= 1e3:  return f"${v/1e3:.0f} thousand"
    return f"${v:.0f}"

# Load normalized/tidy financials generated earlier.
df_fin=pd.read_csv("out/financials_last2y.csv")

# --- SAFE ground truth (no chained assignment) ---
# Given (ticker, year, line_item), locate the most material matching row and normalize its value to base units.
# Returns a tuple: (value_in_base_units, currency_code) or None if not found even via fuzzy match.
def ground_truth_value(df: pd.DataFrame, ticker: str, year: int, line_item: str):
    sub = df.loc[
        (df["ticker"] == ticker) &
        (df["fiscal_year"] == year) &
        (df["line_item"] == line_item)
    ].copy()

    if sub.empty:
        sub = df.loc[
            (df["ticker"] == ticker) &
            (df["fiscal_year"] == year) &
            (df["line_item"].str.contains(re.escape(line_item), case=False, regex=True))
        ].copy()
        if sub.empty:
            return None

    row = (
        sub.assign(abs_val=sub["value"].abs())   # <- no direct sub["abs_val"] write
           .nlargest(1, "abs_val")
           .iloc[0]
    )
    v_norm, u_norm = normalize_value(row["value"], row["unit"])
    return v_norm, u_norm

# Relative closeness checker with default 2% tolerance; guards against zero by using max(1, |a|, |b|).
def rel_close(a: float, b: float, tol: float = 0.02) -> bool:
    # relative tolerance (2% default), protects against zero with +1
    denom = max(1.0, abs(a), abs(b))
    return abs(a - b) / denom <= tol

# Core evaluation:
# - For each Q/A pair:
#   1) Call your answerer to produce a prediction.
#   2) Parse numeric value from structured result or from free text via regex.
#   3) Compare against normalized GT with relative tolerance for Accuracy@1.
#   4) Compute retrieval Recall@k by checking if GT ticker/year/(statement) appears among top-k contexts.
def eval_accuracy_and_recall(qa_list, k_list=(1,3,5), verbose=False):
    correct = 0
    total_numeric = 0
    coverage = 0
    recalls = {k: 0 for k in k_list}
    n = 0

    for p in qa_list:
        n += 1
        t = p["ticker"]; y = int(p["year"]); li = p["line_item"]
        # Our metric detector expects generic names; not required for accuracy since we use lookup in answerer.
        res = answer_query(p["question"], k_ctx=max(k_list), alpha=0.6)

        # Ground truth numeric
        gt = ground_truth_value(df_fin, t, y, li)
        if gt is None:
            if verbose:
                print(f"[skip] no GT for {t} {y} {li}")
            continue
        gt_val, gt_unit = gt

        # Predicted numeric
        pred_has_num = False
        pred_val = None
        if res["source"] == "structured_lookup" and "value_norm" in res:
            pred_val = float(res["value_norm"])
            pred_has_num = True
        else:
            # Try to parse a $X (million/billion/thousand) pattern from the text
            m = re.search(r"\$([0-9]+(?:\.[0-9]+)?)\s*(thousand|million|billion|trillion)?", res["answer"], flags=re.I)
            if m:
                x = float(m.group(1))
                unit_word = (m.group(2) or "").lower()
                mul = {"thousand":1e3,"million":1e6,"billion":1e9,"trillion":1e12}.get(unit_word, 1.0)
                pred_val = x * mul
                pred_has_num = True

        if pred_has_num:
            total_numeric += 1
            if rel_close(pred_val, gt_val, tol=0.02):
                correct += 1
        else:
            # no numeric prediction—counts against accuracy coverage
            pass

        # Retrieval recall@k: if any top-k context matches ticker & year & statement of GT row
        matched_stmt = None
        # determine expected statement from df_fin
        row_stmt = df_fin[(df_fin["ticker"]==t) & (df_fin["fiscal_year"]==y) & (df_fin["line_item"]==li)]
        if row_stmt.empty:
            # fuzzy again
            row_stmt = df_fin[(df_fin["ticker"]==t) & (df_fin["fiscal_year"]==y)]
            row_stmt = row_stmt[row_stmt["line_item"].str.contains(re.escape(li), case=False, regex=True)]
        exp_stmt = None if row_stmt.empty else row_stmt.iloc[0]["statement"]

        # get contexts used for answering
        ctxs = retrieve_contexts(p["question"], k_ctx=max(k_list), alpha=0.6)

        for k in k_list:
            topk = ctxs[:k]
            hit = False
            for c in topk:
                if c.get("ticker")==t and int(c.get("year",-1))==y:
                    # if we know the expected statement, require it; else just ticker+year match
                    if exp_stmt:
                        if c.get("statement")==exp_stmt:
                            hit = True; break
                    else:
                        hit = True; break
            if hit:
                recalls[k] += 1

        # optional logs
        if verbose and (res["source"] != "structured_lookup"):
            print(f"[fallback] {p['question']} -> {res['answer']}")

    acc_at_1 = (correct / total_numeric) if total_numeric else 0.0
    recall_stats = {k: recalls[k]/n for k in k_list} if n else {k: 0.0 for k in k_list}
    return {
        "n_questions": n,
        "numeric_coverage": total_numeric / n if n else 0.0,
        "accuracy_at_1": acc_at_1,
        "recall_at_k": recall_stats
    }

# Run evaluation across the loaded Q/A pairs and print a compact JSON summary.
metrics = eval_accuracy_and_recall(qa_pairs_eval, k_list=(1,3,5), verbose=False)
print(json.dumps(metrics, indent=2))


{
  "n_questions": 376,
  "numeric_coverage": 0.6223404255319149,
  "accuracy_at_1": 0.8846153846153846,
  "recall_at_k": {
    "1": 0.6170212765957447,
    "3": 0.6223404255319149,
    "5": 0.6223404255319149
  }
}


# 2.6 Guardrail Implementation

## 2.6.1 Config & helpers (banned terms, PII masking, allowed tickers)

In [28]:
# --- Guardrail configuration & helpers ---
import re
import pandas as pd

# Your curated universe
ALLOWED_TICKERS = set(['AAGH', 'ADP', 'AEHR', 'AFRM'])

# Very compact "clearly-bad" input patterns (POC-grade; keep conservative)
BANNED_PATTERNS = [
    r"\bkill\b", r"\bsuicide\b", r"\bviolence\b",
    r"\bhack(?:ing)?\b", r"\bmalware\b", r"\bransomware\b",
]

# Finance keywords: require at least one if no ticker is detected
FINANCE_KEYWORDS = [
    "revenue","income","profit","loss","assets","liabilities","equity",
    "cash flow","operating cash","capex","balance sheet","income statement",
    "dividend","ebit","ebitda","net income","total assets","total liabilities"
]

# Simple PII masking (emails, phone-like, SSN-like) - best-effort only
def mask_pii(text: str) -> str:
    t = text
    t = re.sub(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", "[email]", t)
    t = re.sub(r"\b(?:\+?\d[\d \-\(\)]{7,}\d)\b", "[phone]", t)
    t = re.sub(r"\b\d{3}-\d{2}-\d{4}\b", "[ssn]", t)   # US SSN pattern
    return t


## 2.6.2 Input guard: validate_query(query)

In [29]:
def validate_query(query: str) -> dict:
    """
    Returns:
      {
        ok: bool,
        sanitized_query: str,  # PII-masked
        reason: str | None,
        parsed_ticker: str | None,
        parsed_year: int | None
      }
    """
    q_raw = (query or "").strip()
    if not q_raw:
        return {"ok": False, "sanitized_query": "", "reason": "empty query", "parsed_ticker": None, "parsed_year": None}

    # mask PII before doing anything else
    q = mask_pii(q_raw)

    # hard-ban obvious harmful intents (keep this narrow)
    for pat in BANNED_PATTERNS:
        if re.search(pat, q, flags=re.I):
            return {"ok": False, "sanitized_query": q, "reason": "harmful intent detected", "parsed_ticker": None, "parsed_year": None}

    # parse expected entities
    tkr, yr = parse_query(q) if "parse_query" in globals() else (None, None)

    # require a supported ticker OR a finance keyword (keeps PoC focused)
    has_fin_kw = any(kw in q.lower() for kw in FINANCE_KEYWORDS)
    if tkr is None and not has_fin_kw:
        return {"ok": False, "sanitized_query": q, "reason": "no ticker or finance keyword found", "parsed_ticker": None, "parsed_year": None}

    # if a ticker is present, require it to be in the allowed set
    if tkr is not None and tkr not in ALLOWED_TICKERS:
        return {"ok": False, "sanitized_query": q, "reason": f"unsupported ticker: {tkr}", "parsed_ticker": tkr, "parsed_year": yr}

    return {"ok": True, "sanitized_query": q, "reason": None, "parsed_ticker": tkr, "parsed_year": yr}


## 2.6.3 Output guard: verify numbers vs. ground-truth & flag risky cases

In [30]:
# Reuse your normalization helpers
def extract_numeric_from_text(text: str):
    """
    Finds the first '$X [thousand/million/billion/trillion]?' pattern and returns a base-USD float, else None.
    """
    m = re.search(r"\$([0-9]+(?:\.[0-9]+)?)\s*(thousand|million|billion|trillion)?", text, flags=re.I)
    if not m:
        return None
    x = float(m.group(1))
    mul = {"thousand":1e3,"million":1e6,"billion":1e9,"trillion":1e12}.get((m.group(2) or "").lower(), 1.0)
    return x * mul

def ground_truth_value(df: pd.DataFrame, ticker: str, year: int, metric_or_line: str):
    """
    Looks up GT value for either a known metric (via your lookup_metric),
    or a line_item string contained in df_fin.
    Returns (value_norm) or None.
    """
    # Try metric path first
    if metric_or_line in (METRIC_PATTERNS.keys()):
        hit = lookup_metric(df, ticker, year, metric_or_line)
        if hit: return float(hit["value_norm"])

    # Fallback: treat as a line_item
    sub = df.loc[(df["ticker"]==ticker) & (df["fiscal_year"]==year) &
                 (df["line_item"].str.contains(re.escape(metric_or_line), case=False, regex=True))].copy()
    if sub.empty:
        return None
    row = (sub.assign(abs_val=sub["value"].abs())
             .nlargest(1, "abs_val").iloc[0])
    v_norm, _ = normalize_value(row["value"], row["unit"])
    return float(v_norm)

def verify_output(answer_obj: dict, tol: float = 0.02) -> dict:
    """
    Adds guardrail fields:
      - flagged: bool
      - flag_reason: str | None
      - confidence: float (crude heuristic)
    """
    flagged = False
    reason  = None
    conf    = 0.5

    # Trust structured lookup more than RAG
    if answer_obj.get("source") == "structured_lookup":
        conf = 0.9
        # sanity: re-compare with GT in case of unexpected mismatch
        t, y = answer_obj.get("ticker"), answer_obj.get("year")
        label = answer_obj.get("metric")  # metric name or line_item
        pred  = float(answer_obj.get("value_norm", 0.0))
        gt    = ground_truth_value(df_fin, t, y, label) if (t and y and label) else None
        if gt is not None:
            denom = max(1.0, abs(gt), abs(pred))
            if abs(pred - gt) / denom > tol:
                flagged, reason, conf = True, "numeric mismatch vs ground-truth", 0.2
    else:
        # RAG fallback → lower confidence; extract number (if any) & compare
        conf = 0.4
        t, y = answer_obj.get("ticker"), answer_obj.get("year")
        label = answer_obj.get("metric")
        num = extract_numeric_from_text(answer_obj.get("answer",""))
        if (t and y and label and num is not None):
            gt = ground_truth_value(df_fin, t, y, label)
            if gt is None:
                flagged, reason, conf = True, "no ground-truth available for numeric claim", 0.2
            else:
                denom = max(1.0, abs(gt), abs(num))
                if abs(num - gt) / denom > tol:
                    flagged, reason, conf = True, "numeric differs from ground-truth", 0.2
                else:
                    conf = 0.7  # matched
        else:
            # No numeric claim → leave as informational, but mark as low confidence
            reason = "no verifiable number in output" if label else "no structured mapping"
            conf   = 0.4

    out = dict(answer_obj)
    out["flagged"] = flagged
    out["flag_reason"] = reason
    out["confidence"] = float(conf)
    return out


## 2.6.4 Guarded wrapper: answer_query_guarded(query)

In [31]:
def answer_query_guarded(query: str, k_ctx: int = 5, alpha: float = 0.6) -> dict:
    """
    Full pipeline with guardrails:
      1) Input validation (block clearly-bad/irrelevant; mask PII)
      2) Answer (your existing answer_query)
      3) Output verification (flag hallucinations/mismatches)
    """
    # 1) input guard
    v = validate_query(query)
    if not v["ok"]:
        return {
            "blocked": True,
            "reason": v["reason"],
            "sanitized_query": v["sanitized_query"],
        }

    # 2) compute answer using sanitized text (preserves PII masking)
    ans = answer_query(v["sanitized_query"], k_ctx=k_ctx, alpha=alpha)

    # 3) output verification
    verified = verify_output(ans, tol=0.02)
    return {
        "blocked": False,
        "query_sanitized": v["sanitized_query"],
        "parsed_ticker": v["parsed_ticker"],
        "parsed_year": v["parsed_year"],
        "result": verified,
    }


## 2.6.5 Quick smoke tests

In [32]:
tests = [
    # valid
    "ADP 2024 revenue",
    "affirm total assets 2024",
    # valid but with obvious PII to mask
    "email me at alice@example.com: AEHR gross profit 2023",
    # no ticker + no finance keyword → blocked as irrelevant (PoC scope)
    "tell me a joke about penguins",
    # banned
    "how to make ransomware work",
]

for q in tests:
    out = answer_query_guarded(q, k_ctx=5, alpha=0.6)
    print("\n[Q]", q)
    if out["blocked"]:
        print(" -> BLOCKED | reason:", out["reason"])
        print("    sanitized:", out["sanitized_query"])
    else:
        r = out["result"]
        print(" ->", r["answer"])
        print(f"    flagged={r['flagged']} reason={r['flag_reason']} conf={r['confidence']:.2f}")
        print("    citations:", r.get("citations"))



[Q] ADP 2024 revenue
 -> ADP 2024 Revenue: $19.20 billion (USD).
    flagged=False reason=None conf=0.90
    citations: ['ADP__income_statement__2024.txt#chunk0', 'ADP__cash_flow__2024.txt#chunk0', 'ADP__balance_sheet__2024.txt#chunk0']

[Q] affirm total assets 2024
 -> AFRM 2024 Total Assets: $9.52 billion (USD).
    flagged=False reason=None conf=0.90
    citations: ['AFRM__balance_sheet__2024.txt#chunk0', 'AFRM__balance_sheet__2024.txt#chunk1', 'AFRM__balance_sheet__2024.txt#chunk2']

[Q] email me at alice@example.com: AEHR gross profit 2023
 -> AEHR 2023 Gross Profit: $32.75 million (USD).
    flagged=False reason=None conf=0.90
    citations: ['AEHR__income_statement__2023.txt#chunk0', 'AEHR__balance_sheet__2023.txt#chunk0', 'AEHR__cash_flow__2023.txt#chunk0']

[Q] tell me a joke about penguins
 -> BLOCKED | reason: no ticker or finance keyword found
    sanitized: tell me a joke about penguins

[Q] how to make ransomware work
 -> BLOCKED | reason: harmful intent detected
    san

# Implement and document your assigned technique in detail in notebook

# Assigned Technique: **Hybrid Dense+Sparse Retrieval with Cross-Encoder Re-Ranking** — Detailed Notes

**What this is:** A *hybrid* retriever that fuses **dense** semantic search (Sentence-Transformer embeddings + FAISS) with **sparse** lexical search (TF-IDF). The fused candidates are then **re-ranked** using a small **cross-encoder** (CE) trained on MS-MARCO to maximize relevance at the very top of the list.

**Why hybrid?** Pure dense search is robust to paraphrase but can miss exact term matches (ticker symbols, line-item names). Pure sparse search captures exact terms but fails on synonyms or longer contexts. By combining them, we get the *recall* of lexical indices and the *semantic precision* of embeddings.

**How fusion works:** We retrieve the top-K from **both** FAISS (cosine on normalized vectors) and TF-IDF (cosine on sparse vectors). Each score list is **min–max normalized** to [0,1] to remove scale effects; then we compute a convex combination `score = α·dense + (1-α)·sparse`. Empirically, `α≈0.6` balances semantic and lexical signals for finance QA. We over-fetch a pool (e.g., 36) to leave room for the CE to correct ordering mistakes.

**Cross-encoder re-ranking:** A CE scores *(query, passage)* jointly with full attention, giving sharper top-k than bi-encoders. We tokenize each pair, forward through `cross-encoder/ms-marco-MiniLM-L-6-v2`, and sort by the relevant logit/probability. This final step materially improves answerability for short numeric questions where phrasing and local context matter.

**Chunking assumptions:** Upstream, documents are split with a token-aware window (e.g., 100/400 tokens with small overlaps) to preserve coherence and meet model limits. The retriever operates on these chunks and returns metadata (ticker, year, statement, path) for citation.

**Guardrails & tuning tips:**  
- Normalize user queries (lowercase, strip punctuation, keep currency/years).  
- Start with `α∈[0.5,0.7]`, pool size 24–48, CE top-k 5–8.  
- Prefer cosine with **normalized** dense vectors (inner-product in FAISS).  
- Log fused vs CE scores to catch drift; watch recall@k and answer EM/within-tolerance metrics.

This technique is drop-in and model-agnostic: we can swap encoders (e.g., E5) or re-rankers (e.g., cross-encoder-msmarco-mpnet) with the same interface.


# 3. Fine-Tuned Model System Implementation

# 3.1) FT — Q/A Dataset Preparation

**Goal:** Reuse Q/A pairs to build numeric regression targets in **base USD** (integers), then split 80/10/10.

**Why (extra FT detail):** Fine-tuning on structured numeric tasks benefits from canonical targets; we remove unit noise and map to reliable line items.


In [33]:
# --- Prereqs expected 

_UNIT_SCALE_PATTERNS = [
    (r"\b(thousand|k|000s)\b", 1e3),
    (r"\b(million|mn|mm|m)\b", 1e6),
    (r"\b(billion|bn|b)\b",    1e9),
    (r"\b(trillion|tn|t)\b",   1e12),
]

def _detect_currency(unit_str: str) -> str:
    # Extract a currency code when present (default to USD).
    u = (unit_str or "").upper()
    m = re.search(r"\b(USD|EUR|GBP|JPY|CNY|CAD|AUD)\b", u)
    return m.group(1) if m else "USD"

def _detect_scale(unit_str: str) -> float:
    u = (unit_str or "").lower().strip()
    # Parenthetical variants like "USD (thousands)"
    if "thousand" in u or "(thousand" in u: return 1e3
    if "million"  in u or "(million"  in u: return 1e6
    if "billion"  in u or "(billion"  in u: return 1e9
    if "trillion" in u or "(trillion" in u: return 1e12
    # Compact markers (bn, mn, mm, k, etc.)
    for pat, mul in _UNIT_SCALE_PATTERNS:
        if re.search(pat, u):
            return mul
    return 1.0
def normalize_value(value: float, unit_str: str) -> Tuple[float, str]:
    """
    Returns (normalized_value, normalized_unit). Example:
      (19.2, 'USD billion') -> (19_200_000_000, 'USD')
      (4157, 'USD thousands') -> (4_157_000, 'USD')
    """
    mul = _detect_scale(unit_str)
    cur = _detect_currency(unit_str)
    return float(value) * mul, cur

df_fin=pd.read_csv("out/financials_last2y.csv")


# Light-weight numeric parser fallback (only used if strict lookup fails)
SCALE_ALIAS = {
    "k": 1e3, "thousand": 1e3,
    "m": 1e6, "mn": 1e6, "mm": 1e6, "million": 1e6,
    "b": 1e9, "bn": 1e9, "billion": 1e9,
    "t": 1e12, "tn": 1e12, "trillion": 1e12,
}
def extract_numeric_from_text(text: str) -> Optional[float]:
    if not text: return None
    s = text.strip()
    m = re.search(r"\$?\s*([0-9][0-9,]*\.?[0-9]*)\s*([kmbtn]|thousand|million|billion|trillion|mn|mm|bn|tn)?\b", s, re.I)
    if m:
        num = float(m.group(1).replace(",", ""))
        scale = (m.group(2) or "").lower()
        return num * SCALE_ALIAS.get(scale, 1.0)
    m = re.search(r"\b([0-9][0-9,]{2,})\b", s)
    return float(m.group(1).replace(",", "")) if m else None

# --- Q/A load & split ---
QA_PATH = "out/qa_pairs.jsonl"
assert os.path.exists(QA_PATH), f"Missing {QA_PATH} (from Step-1)."

with open(QA_PATH, "r", encoding="utf-8") as f:
    qa_all = [json.loads(line) for line in f]

qa_all = [x for x in qa_all if x.get("question") and x.get("answer")]
random.seed(42); random.shuffle(qa_all)
n = len(qa_all); n_train = max(1, int(0.8*n)); n_val = max(1, int(0.1*n))
train_raw = qa_all[:n_train]; val_raw = qa_all[n_train:n_train+n_val]; test_raw = qa_all[n_train+n_val:]
print(f"[data] total={n} | train={len(train_raw)} | val={len(val_raw)} | test={len(test_raw)}")

# --- Canonical line-item matching rules (strict) ---
CANON_PATTERNS = {
    "revenue":               [r"^revenue$", r"^total revenue$"],
    "cost of revenue":       [r"^cost of revenue$"],
    "gross profit":          [r"^gross profit$"],
    "net income":            [r"^net income$"],
    "operating cash flow":   [r"^net cash from operating activities$", r"^cash provided by operating activities$"],
    "total assets":          [r"^total assets$"],
    "total liabilities":     [r"^total liabilities$"],  # excludes liabilities & equity
    "shareholders equity":   [r"^total shareholders'? equity$", r"^total stockholders'? equity$", r"^shareholders'? equity$"],
    "cash & equivalents":    [r"^cash( and)? cash equivalents$",
                              r"^cash,?\s*cash equivalents$"],
    "capex":                 [r"^capital expenditures$", r"^purchases of property, plant and equipment$"],
}


EXCLUDE_PATTERNS = [r"liabilities\s*&\s*equity", r"liabilities\s*and\s*equity"]

def _exclude_bad_rows(df: pd.DataFrame, item_lower: str) -> pd.DataFrame:
    out = df
    if "liabilities" in item_lower and "equity" not in item_lower:
        bad = "|".join(EXCLUDE_PATTERNS)
        out = out[~out["line_item"].str.contains(bad, case=False, na=False)]
    return out

def _match_best_row(df: pd.DataFrame, wanted: str) -> Optional[pd.Series]:
    if df.empty: return None
    wanted_norm = (wanted or "").strip().lower()
    df2 = _exclude_bad_rows(df, wanted_norm)
    # 1) exact fullmatch on raw string
    m = df2[df2["line_item"].str.fullmatch(re.escape(wanted), case=False, na=False)]
    if not m.empty: return m.assign(abs_val=m["value"].abs()).nlargest(1, "abs_val").iloc[0]
    # 2) canonical synonyms
    for patt in CANON_PATTERNS.get(wanted_norm, []):
        m = df2[df2["line_item"].str.match(patt, case=False, na=False)]
        if not m.empty: return m.assign(abs_val=m["value"].abs()).nlargest(1, "abs_val").iloc[0]
    # 3) word-boundary contains
    patt = r"\b" + re.escape(wanted) + r"\b"
    m = df2[df2["line_item"].str.contains(patt, case=False, na=False)]
    if not m.empty: return m.assign(abs_val=m["value"].abs()).nlargest(1, "abs_val").iloc[0]
    # 4) last resort: loose contains
    m = df2[df2["line_item"].str.contains(re.escape(wanted), case=False, na=False)]
    if not m.empty: return m.assign(abs_val=m["value"].abs()).nlargest(1, "abs_val").iloc[0]
    return None

def canonical_numeric_answer_strict(row: Dict) -> Optional[int]:
    t = row.get("ticker"); y = row.get("year"); li = (row.get("line_item") or "").strip()
    if t is not None and y is not None and li:
        sub = df_fin.loc[(df_fin["ticker"]==t) & (df_fin["fiscal_year"]==int(y))].copy()
        if not sub.empty:
            hit = _match_best_row(sub, li)
            if hit is not None:
                v_norm, _ = normalize_value(hit["value"], hit["unit"])
                return int(round(float(v_norm)))
    # Fallback: parse numeric from provided answer text
    gold_num = extract_numeric_from_text(row.get("answer",""))
    return int(round(gold_num)) if gold_num is not None else None

def build_numeric_pairs_strict(rows: List[Dict]) -> List[Dict]:
    out = []
    for r in rows:
        val = canonical_numeric_answer_strict(r)
        if val is None: continue
        out.append({"q": r["question"], "y": val, "ticker": r.get("ticker"), "year": r.get("year"), "line_item": r.get("line_item")})
    return out

train = build_numeric_pairs_strict(train_raw)
val   = build_numeric_pairs_strict(val_raw)
test  = build_numeric_pairs_strict(test_raw)
print(f"[numeric(strict)] train={len(train)} | val={len(val)} | test={len(test)}")


[data] total=376 | train=300 | val=37 | test=39
[numeric(strict)] train=300 | val=37 | test=39


## 3.1.1 Canonicalization & Signed-Log Targets

**What:** Map noisy line items → canonical keys; compute **signed log10** targets (`z=sign*log10(1+|y/scale|)`).

**Why (extra FT detail):** Stabilizes regression across magnitudes (thousands ↔ trillions) and reduces outlier impact.


In [34]:
# --- Canonicalization, filtering, and log-scaled target building for financial line items ---
# Purpose:
# 1) Map noisy/raw line-item names to a small set of canonical keys for robust modeling.
# 2) Filter out unreliable/low-variance or irrelevant categories to improve regression quality.
# 3) Build augmented questions and log10-signed, per-metric–scaled targets (z) for training.
# Notes:
# - Does not change the raw targets (y); only constructs a transformed target (z) alongside metadata.
# - Uses simple regex heuristics to canonicalize line-item names.

def _norm(s: str) -> str:
    # Normalize whitespace and lowercase for consistent regex checks
    return re.sub(r"\s+", " ", (s or "").strip().lower())

def _canon_key(li: str) -> str:
    # Heuristic mapping of a raw line item string to a canonical metric key
    x = _norm(li)
    # Drop noisy CF events / financing items for regression (issuances, repurchases, etc.)
    if re.search(r"\bcash (from|used in)\b", x) or "repurchase" in x or "issuance of" in x:
        return "drop_cf_event"
    # High-priority exact metrics
    if re.search(r"\bcost of revenue\b", x):                                  return "cost of revenue"
    if re.search(r"\bgross profit\b", x):                                     return "gross profit"
    if re.search(r"\bnet income\b", x) and "starting line" not in x:          return "net income"
    if re.search(r"\b(total )?revenue\b", x):                                 return "revenue"
    if re.search(r"\bnet cash (provided by|from) operating activities\b", x): return "operating cash flow"
    if re.search(r"\bcash (and|&) cash equivalents\b", x) or \
       re.search(r"\bcash,?\s*cash equivalents\b", x):                        return "cash & equivalents"
    if re.search(r"\btotal assets\b", x):                                     return "total assets"
    if re.search(r"\btotal liabilities\b", x) and "equity" not in x:          return "total liabilities"
    if re.search(r"\b(total (shareholders|stockholders)'? equity|shareholders'? equity)\b", x):
        return "shareholders equity"
    if re.search(r"\b(capital expenditures|purchases of property, plant and equipment)\b", x):
        return "capex"
    # Flat/irrelevant for regression here
    if "starting line" in x or x.startswith("shares ("):
        return "drop_noisy"
    # Unknown/uncategorized items
    return "unknown"

# Keep reliable metrics only
# ALLOWED: canonical keys we intend to model; BLOCKLIST: keys to remove outright.
ALLOWED = {
    "revenue", "cost of revenue", "gross profit", "net income",
    "operating cash flow", "total assets", "total liabilities",
    "shareholders equity", "cash & equivalents", "capex"
}
BLOCKLIST = {"drop_cf_event", "drop_noisy", "unknown"}

# Low-variance filter in log space (removes near-constant artifact groups)
# Build a combined DataFrame to compute per-canon_key statistics.
df_all = pd.DataFrame(train + val + test).copy()
df_all["canon_key"] = df_all["line_item"].apply(_canon_key)
tmp = df_all.copy()
# Use log10(1 + |y|) as a robust spread proxy across magnitudes
tmp["log_abs_y"] = np.log10(1.0 + tmp["y"].astype(float).abs())
var = (tmp.groupby("canon_key")["log_abs_y"]
          .agg(mu="mean", sigma="std", n="count")
          .reset_index())
# Threshold for "low variance" groups — tune as needed
LOW_VAR_SIGMA = 0.12
low_var_keys = set(var.loc[var["sigma"].fillna(0.0) < LOW_VAR_SIGMA, "canon_key"])

# Final drop set = (low-variance groups ∪ explicit blocklist) \ ALLOWED (never drop allowed)
DROP_KEYS = (low_var_keys | BLOCKLIST) - ALLOWED
print("[filter] low-variance keys:", sorted(low_var_keys))
print("[filter] drop keys:",        sorted(DROP_KEYS))

def _keep_row(r: dict) -> bool:
    # Keep a row only if its canonical key is allowed and not in DROP_KEYS
    k = _canon_key(r.get("line_item"))
    return (k in ALLOWED) and (k not in DROP_KEYS)

# Filter splits in place (list comprehensions preserve order)
train = [r for r in train if _keep_row(r)]
val   = [r for r in val   if _keep_row(r)]
test  = [r for r in test  if _keep_row(r)]
print(f"[filter] kept -> train={len(train)} | val={len(val)} | test={len(test)}")

# Per-metric scales (USD)
# Used to stabilize targets by scaling values to roughly similar log10 ranges per metric.
ITEM_SCALE = {
    "revenue": 1e9, "cost of revenue": 1e9, "gross profit": 1e9, "net income": 1e9,
    "operating cash flow": 1e9, "total assets": 1e9, "total liabilities": 1e9,
    "shareholders equity": 1e9, "cash & equivalents": 1e9, "capex": 1e6
}

def _augment_question(r: dict) -> str:
    # Add lightweight tags to the question text for the model to leverage structured hints
    t = r.get("ticker") or ""
    y = r.get("year")
    li = (r.get("line_item") or "").strip()
    tag_t = f"[TICKER={t}]" if t else ""
    tag_y = f"[YEAR={int(y)}]" if y is not None else ""
    tag_li = f"[ITEM={li}]" if li else ""
    base_q = r.get("q") or r.get("question")
    return " ".join([tag_t, tag_y, tag_li, base_q]).strip()

def _to_log10_signed_scaled(y_usd: float, scale: float) -> float:
    # Signed log transform: sign(y) * log10(1 + |y|/scale)
    # Keeps negative values distinguishable and compresses heavy tails.
    sign = 1.0 if y_usd >= 0 else -1.0
    return sign * math.log10(1.0 + abs(float(y_usd))/float(scale))

def _canon_for_scale(li: str) -> str:
    # Choose the scale bucket based on canonical key; default to "revenue" scale (billions)
    k = _canon_key(li)
    return k if k in ITEM_SCALE else "revenue"  # default billions

def _build_aug_log_scaled(rows):
    # Construct training rows with:
    # - q: augmented question (with tags)
    # - y: original value
    # - z: signed, log10-scaled target
    # - scale: per-item scale used for z
    # - metadata: ticker, year, line_item, canon_key
    out = []
    for r in rows:
        y  = float(r["y"])
        ck = _canon_for_scale(r.get("line_item") or "")
        sc = ITEM_SCALE[ck]
        q_aug = _augment_question(r)
        z = _to_log10_signed_scaled(y, sc)
        out.append({
            "q": q_aug, "y": y, "z": float(z), "scale": float(sc),
            "ticker": r.get("ticker"), "year": r.get("year"),
            "line_item": r.get("line_item"), "canon_key": ck
        })
    return out

# Build transformed splits used by the model
train_z = _build_aug_log_scaled(train)
val_z   = _build_aug_log_scaled(val)
test_z  = _build_aug_log_scaled(test)

# Quick distribution sanity checks for z
z_list = [x["z"] for x in train_z]
print(f"[rebuild(clean)] train_z={len(train_z)} val_z={len(val_z)} test_z={len(test_z)}")
if z_list:
    print(f"[z stats] mean={np.mean(z_list):.3f} std={np.std(z_list, ddof=1) if len(z_list)>1 else 0.0:.3f} "
          f"min={min(z_list):.3f} max={max(z_list):.3f}")
    print("[sample]", train_z[0] if train_z else "—")


[filter] low-variance keys: ['drop_noisy']
[filter] drop keys: ['drop_cf_event', 'drop_noisy', 'unknown']
[filter] kept -> train=86 | val=11 | test=9
[rebuild(clean)] train_z=86 val_z=11 test_z=9
[z stats] mean=0.304 std=0.606 min=-1.002 max=1.743
[sample] {'q': "[TICKER=AFRM] [YEAR=2024] [ITEM=Total Liabilities] What was AFRM's Total Liabilities in 2024?", 'y': 6787630000.0, 'z': 0.8914053094664443, 'scale': 1000000000.0, 'ticker': 'AFRM', 'year': 2024, 'line_item': 'Total Liabilities', 'canon_key': 'total liabilities'}


## 3.2 Model Selection: Encoder Setup (MiniLM) + Mean-Pool

**Why (extra FT detail):** We reuse a compact **MiniLM** encoder for text → vector; mean-pooling is stable and fast for regression tasks.


In [35]:
# --- Embedding backbone (MiniLM) lazy-load + mean pooling ---
# Purpose:
# - Select DEVICE (GPU if available, else CPU).
# - Lazily initialize the tokenizer/model/hidden size only if they aren't already defined
#   (so re-running the cell doesn't reload weights).
# - Provide a standard attention-masked mean pooling function compatible with HF models.

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # auto-pick CUDA when available

# Load once (reuse if already loaded earlier)
try:
    enc        # HF encoder model (AutoModel)
    tok        # HF tokenizer (AutoTokenizer)
    HIDDEN     # hidden size (embedding dimension)
except NameError:
    MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
    tok = AutoTokenizer.from_pretrained(MODEL_NAME)            # text → token ids
    enc = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE)     # encoder to DEVICE for speed
    HIDDEN = enc.config.hidden_size                             # e.g., 384 for MiniLM-L6-v2

# --- Attention-masked mean pooling ---
# Inputs:
# - last_hidden_state: [batch, seq_len, hidden] from the transformer
# - attention_mask    : [batch, seq_len] with 1 for real tokens, 0 for padding
# Output:
# - Pooled embeddings: [batch, hidden], computed as mean over non-padded tokens
def mean_pool(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1).float()     # [B, L] → [B, L, 1] for broadcast
    summed = (last_hidden_state * mask).sum(dim=1)  # sum embeddings over valid tokens
    counts = mask.sum(dim=1).clamp(min=1e-6)        # avoid divide-by-zero when sequences are empty
    return summed / counts                           # mean = sum / count of valid tokens



# 3.3) Baseline Benchmarking (Pre-Fine-Tuning): Zero-shot kNN Baseline (K=10)

**Why:** A simple non-parametric baseline helps sanity-check before training the MoE model.


In [36]:
@torch.no_grad()  # inference only: no gradients, faster & less memory
def embed_texts(texts, max_len=128, batch=64):
    """
    Encode a list of texts into fixed-size embeddings via the HF encoder.
    Inputs:
      - texts   : List[str] to embed
      - max_len : tokenizer truncation length
      - batch   : batch size for throughput
    Returns:
      - np.ndarray of shape [N, HIDDEN] with float32 embeddings (CPU)
    Assumptions:
      - Globals: tok (AutoTokenizer), enc (AutoModel), DEVICE, HIDDEN
      - mean_pool(...) defined elsewhere for attention-masked averaging
    """
    vecs = []
    for i in range(0, len(texts), batch):
        batch_texts = texts[i:i+batch]
        # Tokenize with padding/truncation and move to DEVICE
        enc_in = tok(batch_texts, padding=True, truncation=True, max_length=max_len, return_tensors="pt").to(DEVICE)
        # Forward pass
        out = enc(**enc_in, return_dict=True)
        # Attention-masked mean pooling → CPU numpy
        pooled = mean_pool(out.last_hidden_state, enc_in["attention_mask"]).cpu().numpy()
        vecs.append(pooled)
    # Stack or return an empty [0, HIDDEN] array if nothing to embed
    return np.vstack(vecs) if vecs else np.zeros((0, HIDDEN), dtype=np.float32)

# --- Build a simple kNN "train bank" from the transformed training set ---
train_q = [r["q"] for r in train_z]                                 # questions/prompts
train_y = np.array([r["y"] for r in train_z], dtype=np.float64)     # original (untransformed) numeric targets
V_train = embed_texts(train_q)                                      # [N_train, HIDDEN] embeddings

def knn_predict(qs, k=5):
    """
    Predict numeric values for queries using cosine-similarity kNN over V_train.
    - Embeds incoming queries
    - Finds top-k nearest neighbors (by cosine)
    - Returns the median of their target values as a robust estimate
    """
    V_q = embed_texts(qs)  # [N_q, HIDDEN]
    preds = []
    for v in V_q:
        # cosine similarity = (A·b) / (||A|| * ||b||)
        sims = V_train @ v / (np.linalg.norm(V_train, axis=1)+1e-9) / (np.linalg.norm(v)+1e-9)
        idx = np.argsort(-sims)[:k]                 # top-k indices by similarity
        preds.append(np.median(train_y[idx]))       # robust aggregator
    return np.array(preds, dtype=np.float64)

def fmt_money(x):
    """
    Pretty-print a numeric value using common financial scales.
    Preserves sign and uses short-scale suffixes.
    """
    sgn = "-" if x < 0 else ""
    x = abs(float(x))
    if x >= 1e12: return f"{sgn}${x/1e12:.2f} trillion"
    if x >= 1e9:  return f"{sgn}${x/1e9:.2f} billion"
    if x >= 1e6:  return f"{sgn}${x/1e6:.2f} million"
    if x >= 1e3:  return f"{sgn}${x/1e3:.2f} thousand"
    return f"{sgn}${x:.0f}"

# --- Quick baseline evaluation on a small test slice ---
def eval_knn(sample, k=10):
    """
    Evaluate kNN on up to k samples:
      - Computes predictions via knn_predict (k fixed at 5 inside)
      - Measures relative error with denom=max(1, |y|, |yhat|)
      - Reports Accuracy@10% and avg per-sample latency (ms)
    """
    sample = sample[:k] if len(sample)>=k else sample
    qs = [r["q"] for r in sample]
    ys = np.array([r["y"] for r in sample], dtype=np.float64)
    t0 = time.time(); yhat = knn_predict(qs, k=5); ms = (time.time()-t0)*1000.0
    denom = np.maximum(1.0, np.maximum(np.abs(ys), np.abs(yhat)))
    rel = np.abs(yhat-ys)/denom
    return {"k": len(sample), "acc@10%": float((rel<=0.10).mean()), "avg_ms": ms/len(sample)}

print("[baseline kNN]", eval_knn(test_z, k=min(10, len(test_z))))


[baseline kNN] {'k': 9, 'acc@10%': 0.4444444444444444, 'avg_ms': 17.927991019354927}


## 3.3.1 Repro pack: set *all* seeds & deterministic backends

In [37]:
# --- Reproducibility utilities ---
# Purpose:
# - Set deterministic seeds and backend configs across Python, NumPy, and PyTorch (CPU/GPU).
# - Reduce nondeterminism from CUDA/cuDNN by toggling PyTorch determinism flags.
# Notes:
# - Strict GPU determinism may impact performance (e.g., cuDNN benchmark disabled).
# - Requires recent PyTorch for torch.use_deterministic_algorithms; guarded for older versions.

SEED = 12345  # pick any integer, keep it fixed

def set_reproducible(seed=SEED):
    # Ensure Python's hash seed and tokenizer parallelism are stable
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    # CUDA/cuBLAS determinism (recommended for strict determinism on GPU)
    if torch.cuda.is_available():
        os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

    # Seed all relevant RNGs
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # cuDNN / PyTorch determinism knobs
    torch.backends.cudnn.deterministic = True  # choose deterministic algorithms when available
    torch.backends.cudnn.benchmark = False     # disable autotuner to avoid nondeterministic choices
    try:
        # Enforce deterministic kernels where possible (warn on fallbacks)
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass  # older torch versions may not support this API

# Apply reproducibility settings immediately
set_reproducible(SEED)

def seed_worker(worker_id):
    # Per-worker seeding for DataLoader workers / multiprocessing jobs
    s = SEED + worker_id
    np.random.seed(s)
    random.seed(s)

# Create a torch.Generator pre-seeded for deterministic sampling (e.g., DataLoader generator)
g = torch.Generator()
g.manual_seed(SEED)


## 3.3.4 Z-Normalization + Loaders with (ticker/metric/year) Indices

**Why:** Normalize targets and feed **context embeddings** (ticker/metric/year) to the model as discrete features.


In [38]:
# --- z normalization (train only) ---
# Compute mean & std of the training z targets for normalization used during model training/inference.
z_train = np.array([r["z"] for r in train_z], dtype=np.float32)
Z_MEAN = float(z_train.mean()) if len(z_train) else 0.0
Z_STD  = float(z_train.std())  if len(z_train) > 1 else 1.0
if Z_STD == 0.0: Z_STD = 1.0
print(f"[z-norm] mean={Z_MEAN:.4f} std={Z_STD:.4f}")

# --- vocabularies ---
# Build categorical vocabularies for tickers, canonical keys, and years across all splits.
TICKER_VOCAB = sorted({r["ticker"] for r in (train_z+val_z+test_z) if r.get("ticker")})
KEY_VOCAB    = sorted({r["canon_key"] for r in (train_z+val_z+test_z)})
YEAR_VOCAB   = sorted({int(r["year"]) for r in (train_z+val_z+test_z) if r.get("year") is not None})

# Index maps for categorical embeddings / lookups.
ticker2id = {t:i for i,t in enumerate(TICKER_VOCAB)}
key2id    = {k:i for i,k in enumerate(KEY_VOCAB)}
year2id   = {y:i for i,y in enumerate(YEAR_VOCAB)}

# Quick visibility of constructed vocabularies.
print("[vocab] tickers:", TICKER_VOCAB)
print("[vocab] keys:", KEY_VOCAB)
print("[vocab] years:", YEAR_VOCAB)

# --- Structured training row with normalized z and categorical indices ---
@dataclass
class RowZNIdxY:
    q: str; z_norm: float; y: float; scale: float
    tick_idx: int; key_idx: int; year_idx: int

# Convert a raw row dict into a RowZNIdxY with normalized z and integer indices.
def row_to_idx_y(r):
    return RowZNIdxY(
        q=r["q"],
        z_norm=(float(r["z"])-Z_MEAN)/Z_STD,
        y=float(r["y"]),
        scale=float(r["scale"]),
        tick_idx=ticker2id.get(r["ticker"], 0) if TICKER_VOCAB else 0,
        key_idx=key2id.get(r["canon_key"], 0),
        year_idx=year2id.get(int(r["year"]), 0) if YEAR_VOCAB and (r.get("year") is not None) else 0,
    )

# --- Torch Dataset wrapper over prebuilt rows ---
class QRegZNIdxY(Dataset):
    def __init__(self, rows): self.rows = [row_to_idx_y(x) for x in rows]
    def __len__(self): return len(self.rows)
    def __getitem__(self, i): return self.rows[i]

# --- Collate function to batch fields into tensors ---
def collate_rows_zn_idx_y(batch):
    texts  = [b.q for b in batch]
    z_norm = torch.tensor([b.z_norm for b in batch], dtype=torch.float32)
    y_usd  = torch.tensor([b.y      for b in batch], dtype=torch.float32)
    scale  = torch.tensor([b.scale  for b in batch], dtype=torch.float32)
    t_idx  = torch.tensor([b.tick_idx for b in batch], dtype=torch.long)
    k_idx  = torch.tensor([b.key_idx  for b in batch], dtype=torch.long)
    y_idx  = torch.tensor([b.year_idx for b in batch], dtype=torch.long)
    return texts, z_norm, y_usd, scale, t_idx, k_idx, y_idx

# --- DataLoaders (single-worker for determinism) ---
BATCH_SIZE = 16
MAX_LEN    = 128
train_loader = DataLoader(
    QRegZNIdxY(train_z), batch_size=BATCH_SIZE, shuffle=False,
    collate_fn=collate_rows_zn_idx_y,     num_workers=0,            # easiest way to avoid worker nondeterminism
    worker_init_fn=seed_worker,
    generator=g, drop_last=False, pin_memory=False
)
val_loader   = DataLoader(
    QRegZNIdxY(val_z),   batch_size=BATCH_SIZE, shuffle=False,
    collate_fn=collate_rows_zn_idx_y,     num_workers=0,            # easiest way to avoid worker nondeterminism
    worker_init_fn=seed_worker,
    generator=g, drop_last=False, pin_memory=False
)


[z-norm] mean=0.3043 std=0.6026
[vocab] tickers: ['AAGH', 'ADP', 'AEHR', 'AFRM']
[vocab] keys: ['cash & equivalents', 'cost of revenue', 'gross profit', 'net income', 'revenue', 'total assets', 'total liabilities']
[vocab] years: [2023, 2024]


## 3.3.5  === Save / Load ID maps (and z-norm + MAX_LEN) ===

In [39]:
VOCABS_PATH = "out/vocabs.json"

def save_vocabs(path=VOCABS_PATH):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    payload = {
        "schema_version": 1,
        "created_at": time.strftime("%Y-%m-%d %H:%M:%S"),
        # training-time constants (helps reproducibility)
        "z_mean": float(Z_MEAN),
        "z_std": float(Z_STD),
        "max_len": int(MAX_LEN),
        # store both lists (ordered vocab) and maps
        "ticker_vocab": list(map(str, TICKER_VOCAB)),
        "key_vocab": list(map(str, KEY_VOCAB)),
        "year_vocab": list(map(int, YEAR_VOCAB)),
        "ticker2id": {str(k): int(v) for k, v in ticker2id.items()},
        "key2id":    {str(k): int(v) for k, v in key2id.items()},
        # JSON keys must be strings → cast year keys to str
        "year2id":   {str(k): int(v) for k, v in year2id.items()},
    }
    with open(path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2, sort_keys=True)
    print(f"[saved vocabs] → {path}")

def load_vocabs(path=VOCABS_PATH):
    global TICKER_VOCAB, KEY_VOCAB, YEAR_VOCAB
    global ticker2id, key2id, year2id
    global Z_MEAN, Z_STD, MAX_LEN

    with open(path, "r", encoding="utf-8") as f:
        p = json.load(f)

    # prefer stored ordered lists; fall back to keys of the maps
    TICKER_VOCAB = p.get("ticker_vocab") or list(p["ticker2id"].keys())
    KEY_VOCAB    = p.get("key_vocab")    or list(p["key2id"].keys())
    YEAR_VOCAB   = p.get("year_vocab")   or [int(k) for k in p["year2id"].keys()]

    ticker2id = {str(k): int(v) for k, v in p["ticker2id"].items()}
    key2id    = {str(k): int(v) for k, v in p["key2id"].items()}
    year2id   = {int(k): int(v) for k, v in p["year2id"].items()}

    # restore z-norm + max_len so inference matches training scaling/tokenization
    Z_MEAN = float(p.get("z_mean", 0.0))
    Z_STD  = float(p.get("z_std", 1.0)) or 1.0
    MAX_LEN = int(p.get("max_len", 128))

    print(f"[loaded vocabs] ← {path}")
    print("[sizes] tickers:", len(TICKER_VOCAB), "keys:", len(KEY_VOCAB), "years:", len(YEAR_VOCAB))
save_vocabs()   # writes out/vocabs.json


[saved vocabs] → out/vocabs.json


# 3.4 & 3.5) MiniLM + **MoE** (Mixture-of-Experts) — Fine-Tuning Head

**What (extra Model/MoE detail):**
- We **freeze** MiniLM (except last block) and add **discrete embeddings** for *(ticker, metric, year)*.
- A lightweight **MoE adapter** (multiple low-rank experts + gating) learns metric-specific transformations.
- Head predicts normalized **z**; we train with SmoothL1 in z-space.

**Why:** MoE lets different experts specialize (e.g., assets vs income), often improving numeric generalization without a huge model.


In [40]:
# Ensure deterministic behavior before training (sets seeds, backend flags, etc.)
set_reproducible(SEED)

# =========================
# Mixture-of-Experts (MoE) building blocks
# =========================

class LoRAExpert(nn.Module):
    """Tiny low-rank expert (down->GELU->up)."""
    def __init__(self, d_model: int, r: int = 32):
        super().__init__()
        # Low-rank bottleneck: d_model → r → d_model (no bias for simplicity)
        self.down = nn.Linear(d_model, r, bias=False)
        self.up   = nn.Linear(r, d_model, bias=False)
        self.act  = nn.GELU()
    def forward(self, h): return self.up(self.act(self.down(h)))  # apply down-projection, nonlinearity, up-projection

class MoEAdapter(nn.Module):
    def __init__(self, d_model: int, num_experts: int = 6, r: int = 64):
        super().__init__()
        # A small pool of LoRA experts composing the adapter
        self.experts = nn.ModuleList([LoRAExpert(d_model, r=r) for _ in range(num_experts)])
        # Dense gate produces a score per expert; softmax will convert to mixture weights
        self.gate = nn.Sequential(nn.Linear(d_model, d_model//2), nn.Tanh(), nn.Linear(d_model//2, num_experts))
        self.drop = nn.Dropout(0.1)  # regularization on the residual delta
    def forward(self, h):
        gate = torch.softmax(self.gate(h), dim=-1)              # [B,K] mixture weights over experts
        expert_outs = torch.stack([e(h) for e in self.experts], dim=1)  # [B,K,H] expert outputs
        delta = (gate.unsqueeze(-1) * expert_outs).sum(dim=1)   # [B,H] weighted sum across experts
        return self.drop(delta), gate                           # return residual delta and (optionally) gate for introspection

class MiniLM_MoE_Regressor_Z_EmbY(nn.Module):
    """MiniLM (frozen except last block) + [ticker|key|year] emb + proj → MoE → head → z_pred"""
    def __init__(self, encoder, tokenizer, d_model: int,
                 num_tickers: int, num_keys: int, num_years: int,
                 tdim=16, kdim=8, ydim=8, num_experts=6, r=64):
        super().__init__()
        # Keep references to the backbone encoder/tokenizer
        self.encoder = encoder; self.tokenizer = tokenizer
        # Small categorical embeddings for (ticker, canon_key, year)
        self.emb_t = nn.Embedding(max(1,num_tickers), tdim)
        self.emb_k = nn.Embedding(max(1,num_keys),   kdim)
        self.emb_y = nn.Embedding(max(1,num_years),  ydim)
        # Project concatenated [pooled_text | t | k | y] back to d_model hidden size
        self.proj  = nn.Linear(d_model + tdim + kdim + ydim, d_model)
        # Mixture-of-Experts adapter on top of the projected representation
        self.moe   = MoEAdapter(d_model=d_model, num_experts=num_experts, r=r)
        # Prediction head in z-space (regression to a single scalar)
        self.head  = nn.Sequential(nn.Linear(d_model, d_model//2), nn.GELU(), nn.Linear(d_model//2, 1))
        # Freeze the entire encoder by default (we will selectively unfreeze the last block below)
        for p in self.encoder.parameters(): p.requires_grad = False

    def forward(self, texts, tick_idx, key_idx, year_idx, max_len=128):
        # Tokenize on-the-fly; keep ops on DEVICE
        enc_in = self.tokenizer(texts, padding=True, truncation=True, max_length=max_len, return_tensors="pt").to(DEVICE)
        with torch.no_grad():  # respect frozen encoder (except potentially un-frozen last block)
            out = self.encoder(**enc_in, return_dict=True)
            pooled = mean_pool(out.last_hidden_state, enc_in["attention_mask"])  # [B,H] pooled sentence embedding
        # Look up categorical embeddings; gracefully handle empty vocab (num_embeddings==0)
        et = self.emb_t(tick_idx.to(DEVICE)) if self.emb_t.num_embeddings>0 else torch.zeros(pooled.size(0),0,device=DEVICE)
        ek = self.emb_k(key_idx.to(DEVICE))
        ey = self.emb_y(year_idx.to(DEVICE)) if self.emb_y.num_embeddings>0 else torch.zeros(pooled.size(0),0,device=DEVICE)
        # Concatenate pooled text + metadata embeddings
        h  = torch.cat([pooled, et, ek, ey], dim=-1)
        h  = self.proj(h)                       # align dims
        delta, _ = self.moe(h)                  # adapter residual
        z_pred = self.head(h + delta).squeeze(-1)  # residual connection into head; output shape [B]
        return {"z_pred": z_pred}

# =========================
# Initialize model
# =========================
moe_z = MiniLM_MoE_Regressor_Z_EmbY(
    enc, tok, d_model=HIDDEN,
    num_tickers=len(TICKER_VOCAB), num_keys=len(KEY_VOCAB), num_years=len(YEAR_VOCAB),
    tdim=16, kdim=8, ydim=8, num_experts=6, r=64
).to(DEVICE)

# =========================
# Partially unfreeze last MiniLM block (light FT for better adaptation)
# =========================
last_layer_params = []
if hasattr(enc, "encoder") and hasattr(enc.encoder, "layer"):
    for p in enc.encoder.layer[-1].parameters():
        p.requires_grad = True
        last_layer_params.append(p)
    print("[unfreeze] last MiniLM block trainable params:", sum(p.numel() for p in last_layer_params))
else:
    print("[warn] could not unfreeze last block")

# =========================
# Optimizer setup (separate LR for head+MoE vs last transformer block)
# =========================
moe_head_params = list(moe_z.moe.parameters()) + list(moe_z.head.parameters()) \
                + list(moe_z.emb_t.parameters()) + list(moe_z.emb_k.parameters()) \
                + list(moe_z.emb_y.parameters()) + list(moe_z.proj.parameters())
for p in moe_head_params: p.requires_grad = True  # ensure trainable

param_groups = [
    {"params": moe_head_params,   "lr": 2e-3, "weight_decay": 0.01},  # faster on new layers
    {"params": last_layer_params, "lr": 2e-4, "weight_decay": 0.01},  # conservative on backbone last block
]
opt = torch.optim.AdamW(param_groups)

# =========================
# Train / Eval loop in z-space with early stopping
# =========================
def run_epoch_norm(model, loader, train=True):
    model.train() if train else model.eval()
    total, n = 0.0, 0
    for texts, z_norm, _, _, t_idx, k_idx, y_idx in loader:
        z_norm = z_norm.to(DEVICE)
        if train: opt.zero_grad()
        out = model(texts, t_idx, k_idx, y_idx, max_len=MAX_LEN)
        loss = nn.SmoothL1Loss()(out["z_pred"], z_norm)  # robust L1-ish loss in normalized z-space
        if train:
            loss.backward()
            torch.nn.utils.clip_grad_norm_([p for p in model.parameters() if p.requires_grad], 1.0)  # avoid exploding grads
            opt.step()
        total += loss.item(); n += 1
    return total/max(1,n)

# --- Patch: robust checkpointing for your 3.5 loop ---
# Paths for saving best/last checkpoints; directory is created if missing.
CKPT_DIR   = "out"
CKPT_BEST  = os.path.join(CKPT_DIR, "minilm_moe_best.pt")   # best-by-val
CKPT_LAST  = os.path.join(CKPT_DIR, "minilm_moe_last.pt")   # last epoch (for debugging)
os.makedirs(CKPT_DIR, exist_ok=True)

# Training schedule + early stopping hyperparameters
EPOCHS   = 60
patience = 30
min_delta = 1e-4  # minimal improvement to reset patience

best_val = float("inf")
best_ep  = -1
bad      = 0

def save_ckpt_best(model, path, epoch, val_loss):
    # Save best checkpoint with epoch and val_loss for provenance
    torch.save({
        "epoch": epoch,
        "val_loss": float(val_loss),
        "model_state_dict": model.state_dict(),
    }, path)

def save_ckpt_last(model, path, epoch, val_loss):
    # Save every epoch’s last checkpoint (useful for debugging/resuming)
    torch.save({
        "epoch": epoch,
        "val_loss": float(val_loss),
        "model_state_dict": model.state_dict(),
    }, path)

# Main training loop with validation and early stopping
for ep in range(1, EPOCHS+1):
    t0 = time.time()
    tr = run_epoch_norm(moe_z, train_loader, train=True)   # one pass over training set
    vl = run_epoch_norm(moe_z, val_loader,   train=False)  # validation loss
    dt = time.time() - t0

    # Always keep a "last" checkpoint (optional but handy)
    save_ckpt_last(moe_z, CKPT_LAST, ep, vl)

    # Check for validation improvement against best (with min_delta margin)
    improved = (vl < best_val - min_delta)
    if improved:
        best_val = vl
        best_ep  = ep
        bad      = 0
        save_ckpt_best(moe_z, CKPT_BEST, ep, vl)  # persist best
    else:
        bad += 1

    print(f"[ep {ep}/{EPOCHS}] train={tr:.4f} | val={vl:.4f} | "
          f"{'*BEST*' if improved else ''} | {dt:.1f}s | no-improve={bad}/{patience}")

    # Early stopping when patience exceeded
    if bad >= patience:
        print(f"[early-stop] best ep={best_ep} val={best_val:.6f}")
        break

# --- IMPORTANT: reload BEST into memory so subsequent evals use best weights ---
if os.path.exists(CKPT_BEST):
    state = torch.load(CKPT_BEST, map_location=DEVICE)
    moe_z.load_state_dict(state["model_state_dict"])
    moe_z.eval()
    print(f"[restored-best] ep={state['epoch']} val={state['val_loss']:.6f} from {CKPT_BEST}")
else:
    print("[warn] best checkpoint not found; using last-epoch weights")


[unfreeze] last MiniLM block trainable params: 1774464
[ep 1/60] train=0.3075 | val=0.3937 | *BEST* | 2.1s | no-improve=0/30
[ep 2/60] train=0.1887 | val=0.2013 | *BEST* | 2.1s | no-improve=0/30
[ep 3/60] train=0.0964 | val=0.0966 | *BEST* | 2.1s | no-improve=0/30
[ep 4/60] train=0.0873 | val=0.0692 | *BEST* | 2.1s | no-improve=0/30
[ep 5/60] train=0.0695 | val=0.0325 | *BEST* | 2.1s | no-improve=0/30
[ep 6/60] train=0.0624 | val=0.0181 | *BEST* | 2.2s | no-improve=0/30
[ep 7/60] train=0.0328 | val=0.0307 |  | 2.1s | no-improve=1/30
[ep 8/60] train=0.0633 | val=0.0337 |  | 2.1s | no-improve=2/30
[ep 9/60] train=0.0403 | val=0.0322 |  | 2.1s | no-improve=3/30
[ep 10/60] train=0.0337 | val=0.0498 |  | 2.1s | no-improve=4/30
[ep 11/60] train=0.0376 | val=0.0290 |  | 2.0s | no-improve=5/30
[ep 12/60] train=0.0252 | val=0.0094 | *BEST* | 2.1s | no-improve=0/30
[ep 13/60] train=0.0138 | val=0.0075 | *BEST* | 2.1s | no-improve=0/30
[ep 14/60] train=0.0116 | val=0.0126 |  | 2.1s | no-improve=1

## 3.5.1 Loading best model from the path

In [41]:
# --- Minimal: load best checkpoint by absolute/relative path ---
# Tip: keep PATH relative to your project root, or pass an absolute path when deploying.
PATH = r"out/minilm_moe_best.pt"  # or an absolute path

# (Rebuild encoder/tokenizer exactly as in training)
# IMPORTANT: The encoder/tokenizer variables (enc, tok) must be instantiated exactly
# like during training (same MODEL_NAME, same tokenizer params). For example:
#   enc = AutoModel.from_pretrained(MODEL_NAME)
#   tok = AutoTokenizer.from_pretrained(MODEL_NAME)
# The hidden size must match the checkpointed model head input.
d_model = enc.config.hidden_size  # HIDDEN used at train time



# =========================
# MoE building blocks
# =========================

class LoRAExpert(nn.Module):
    """Tiny low-rank expert (down->GELU->up)."""
    def __init__(self, d_model: int, r: int = 32):
        super().__init__()
        # Low-rank bottleneck; no bias for simplicity/determinism
        self.down = nn.Linear(d_model, r, bias=False)
        self.up   = nn.Linear(r, d_model, bias=False)
        self.act  = nn.GELU()
    def forward(self, h): return self.up(self.act(self.down(h)))  # [B,H] → [B,r] → GELU → [B,H]

class MoEAdapter(nn.Module):
    def __init__(self, d_model: int, num_experts: int = 6, r: int = 64):
        super().__init__()
        # Pool of LoRA experts; all see the same input representation
        self.experts = nn.ModuleList([LoRAExpert(d_model, r=r) for _ in range(num_experts)])
        # Dense gate predicts mixture weights over experts
        self.gate = nn.Sequential(nn.Linear(d_model, d_model//2), nn.Tanh(), nn.Linear(d_model//2, num_experts))
        self.drop = nn.Dropout(0.1)  # light regularization on residual
    def forward(self, h):
        gate = torch.softmax(self.gate(h), dim=-1)              # [B,K] soft mixture weights
        expert_outs = torch.stack([e(h) for e in self.experts], dim=1)  # [B,K,H] expert outputs
        delta = (gate.unsqueeze(-1) * expert_outs).sum(dim=1)   # [B,H] weighted sum across experts
        return self.drop(delta), gate                           # return residual delta + (optional) gate for analysis


class MiniLM_MoE_Regressor_Z_EmbY(nn.Module):
    """MiniLM (frozen except last block) + [ticker|key|year] emb + proj → MoE → head → z_pred"""
    def __init__(self, encoder, tokenizer, d_model: int,
                 num_tickers: int, num_keys: int, num_years: int,
                 tdim=16, kdim=8, ydim=8, num_experts=6, r=64):
        super().__init__()
        # Backbone (must match training time); tokenizer used for on-the-fly batching
        self.encoder = encoder; self.tokenizer = tokenizer
        # Small categorical embeddings for metadata
        self.emb_t = nn.Embedding(max(1,num_tickers), tdim)
        self.emb_k = nn.Embedding(max(1,num_keys),   kdim)
        self.emb_y = nn.Embedding(max(1,num_years),  ydim)
        # Project pooled text + metadata embeddings back to d_model
        self.proj  = nn.Linear(d_model + tdim + kdim + ydim, d_model)
        # MoE adapter on top of the projected representation
        self.moe   = MoEAdapter(d_model=d_model, num_experts=num_experts, r=r)
        # Regression head in z-space (outputs a single scalar)
        self.head  = nn.Sequential(nn.Linear(d_model, d_model//2), nn.GELU(), nn.Linear(d_model//2, 1))
        # Freeze full encoder; at train time you may have unfrozen the last block only
        for p in self.encoder.parameters(): p.requires_grad = False

    def forward(self, texts, tick_idx, key_idx, year_idx, max_len=128):
        # Tokenize & encode (inference-time; encoder is frozen)
        enc_in = self.tokenizer(texts, padding=True, truncation=True, max_length=max_len, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            out = self.encoder(**enc_in, return_dict=True)
            pooled = mean_pool(out.last_hidden_state, enc_in["attention_mask"])  # [B,H] pooled sentence embedding
        # Metadata embeddings; handle empty vocabs gracefully
        et = self.emb_t(tick_idx.to(DEVICE)) if self.emb_t.num_embeddings>0 else torch.zeros(pooled.size(0),0,device=DEVICE)
        ek = self.emb_k(key_idx.to(DEVICE))
        ey = self.emb_y(year_idx.to(DEVICE)) if self.emb_y.num_embeddings>0 else torch.zeros(pooled.size(0),0,device=DEVICE)
        # Concatenate pooled text + metadata embeddings, project, then adapt with MoE residual
        h  = torch.cat([pooled, et, ek, ey], dim=-1)
        h  = self.proj(h)
        delta, _ = self.moe(h)
        z_pred = self.head(h + delta).squeeze(-1)  # residual connection into head; shape [B]
        return {"z_pred": z_pred}


# Instantiate the MoE model skeleton with the same dims/vocabs as training
# Requirements:
# - DEVICE, TICKER_VOCAB, KEY_VOCAB, YEAR_VOCAB, mean_pool must exist (from prior cells)
moe_z = MiniLM_MoE_Regressor_Z_EmbY(
    enc, tok, d_model=d_model,
    num_tickers=len(TICKER_VOCAB), num_keys=len(KEY_VOCAB), num_years=len(YEAR_VOCAB),
    tdim=16, kdim=8, ydim=8, num_experts=6, r=64
).to(DEVICE)

# Load checkpoint weights
# torch.load(..., map_location=DEVICE) keeps tensors on the correct device.
ckpt = torch.load(PATH, map_location=DEVICE)
# Support both formats: {"model_state_dict": ...} or raw state_dict
state = ckpt.get("model_state_dict", ckpt)  # works if file stores either dict or raw state_dict
moe_z.load_state_dict(state, strict=True)   # set strict=False only if you changed the code structure
moe_z.eval()                                # switch to eval mode for inference
print(f"Loaded best from {PATH} (saved at epoch {ckpt.get('epoch', '?')}, val={ckpt.get('val_loss', '?')})")


Loaded best from out/minilm_moe_best.pt (saved at epoch 49, val=0.0007196370279416442)


 ## --- Use the fine-tuned MiniLM-MoE model for predictions in 3.7 ---

In [42]:
# 0) sanity: we expect these to exist from earlier steps
assert 'moe_z' in globals(), "moe_z (fine-tuned MiniLM-MoE) not found"
assert 'tok' in globals(), "Tokenizer 'tok' not found"
assert 'DEVICE' in globals(), "DEVICE not set"
assert 'KEY_VOCAB' in globals() and 'TICKER_VOCAB' in globals() and 'YEAR_VOCAB' in globals(), \
       "KEY_VOCAB/TICKER_VOCAB/YEAR_VOCAB missing"
assert 'train_z' in globals(), "train_z missing (for z-scale stats)"

# 1) id maps (build if you don't already have them)
TICKER2ID = {t: i for i, t in enumerate(KEY_VOCAB if False else TICKER_VOCAB)}  # keep structure explicit
KEY2ID    = {k: i for i, k in enumerate(KEY_VOCAB)}
YEAR2ID   = {int(y): i for i, y in enumerate(YEAR_VOCAB)}

# 2) signed-log transforms (must match your training)
def to_log10_signed(y: float) -> float:
    return (1.0 if y >= 0 else -1.0) * np.log10(1.0 + abs(float(y)))

def from_log10_signed(z: float) -> float:
    # inverse of signed log10(1+|y|)
    return np.sign(z) * (10.0 ** abs(z) - 1.0)

# 3) per-metric z normalization stats μ, σ (compute if not already present)
if 'Z_STATS' not in globals():
    Z_STATS = {}
    dfz = pd.DataFrame(train_z)  # fields: q, y, z, canon_key, ticker, year
    for key, g in dfz.groupby('canon_key'):
        # use stored z if available; else recompute
        if 'z' in g.columns:
            zs = g['z'].astype(float).values
        else:
            ys = g['y'].astype(float).values
            zs = np.array([to_log10_signed(v) for v in ys], dtype=np.float64)
        mu = float(np.mean(zs))
        sg = float(np.std(zs))
        if not np.isfinite(sg) or sg < 1e-6: sg = 1.0
        Z_STATS[key] = {'mu': mu, 'sigma': sg}

def _inv_denorm_z(z_norm: float, key: str) -> float:
    st = Z_STATS.get(key, {'mu': 0.0, 'sigma': 1.0})
    mu, sg = float(st['mu']), float(st['sigma'])
    if not np.isfinite(sg) or sg < 1e-6: sg = 1.0
    return float(z_norm) * sg + mu

# 4) MoE inference → USD
@torch.no_grad()
def predict_usd_from_row_MoE(r: dict, max_len: int = 128) -> float:
    """
    r: dict with fields {'q', 'ticker', 'canon_key', 'year', 'y' (optional)}
    returns: predicted value in USD (float)
    """
    moe_z.eval()

    # text (already augmented in your 3.1: [TICKER=...] [YEAR=...] [ITEM=...] ...)
    text = r.get('q') or r.get('question')  # fall back just in case

    # indices for embeddings
    t_idx = torch.tensor([TICKER2ID.get(r.get('ticker'), 0)], dtype=torch.long, device=DEVICE)
    k_idx = torch.tensor([KEY2ID.get(r.get('canon_key'), 0)], dtype=torch.long, device=DEVICE)
    try:
        yid = YEAR2ID.get(int(r.get('year')), 0)
    except Exception:
        yid = 0
    y_idx = torch.tensor([yid], dtype=torch.long, device=DEVICE)

    # forward → normalized z
    out = moe_z([text], t_idx, k_idx, y_idx, max_len=max_len)
    z_norm_pred = out["z_pred"].view(-1)[0].detach().cpu().item()

    # denormalize z and invert to USD
    z_pred = _inv_denorm_z(z_norm_pred, r.get('canon_key'))
    y_hat  = from_log10_signed(z_pred)
    return float(y_hat)

# 5) Plug it into your existing 3.7 pipeline by overriding the old hook
predict_usd_from_row_embY = predict_usd_from_row_MoE
print("[ok] 3.7 evaluator will now use MiniLM-MoE (moe_z) for predictions.")


[ok] 3.7 evaluator will now use MiniLM-MoE (moe_z) for predictions.


# 3.6) Guardrail Implementation (Input + Output)

**Why (extra FT detail):** Clamp predictions to reasonable per-metric ranges; flag out-of-distribution values; basic input checks keep the demo robust.

- Input: require [TICKER=...][YEAR=...][ITEM=...] and basic sanity checks
- Output: clamp to train min/max with a small margin; flag if outside


In [43]:
import datetime as dt
ALLOWED_ITEMS = set(KEY_VOCAB)
CURR_YEAR = dt.datetime.now().year

# ---------- Input guard ----------
_TAG_RE = re.compile(r"\[(?P<k>ticker|year|item)\s*=\s*(?P<v>[^\]]+)\]", re.I)

def _parse_tags(q: str):
    tags = {}
    for m in _TAG_RE.finditer(q):
        k = m.group("k").lower().strip()
        v = m.group("v").strip()
        tags[k] = v
    return tags

def _canon_item(s: str) -> str:
    return s.strip().lower().replace(" ", "_")

def validate_query(q: str) -> (bool, str):
    """Require [TICKER], [YEAR], [ITEM]; basic sanity checks."""
    qs = q.lower()

    # ultra-simple harmful content blocker
    if "ransomware" in qs or "malware" in qs:
        return False, "harmful intent"

    # require tags
    tags = _parse_tags(q)
    if not {"ticker","year","item"} <= set(tags):
        return False, "missing finance tags (TICKER/YEAR/ITEM)"

    # ticker check (if vocab present)
    t = tags["ticker"].upper()
    if TICKER_VOCAB and t not in TICKER_VOCAB:
        return False, f"unknown ticker '{t}'"

    # year check
    try:
        y = int(tags["year"])
    except ValueError:
        return False, "year must be an integer"
    if y < 1980 or y > CURR_YEAR + 1:
        return False, f"year out of range: {y}"

    # item check
    k = _canon_item(tags["item"])
    if ALLOWED_ITEMS and k not in ALLOWED_ITEMS:
        return False, f"unknown item '{k}'"

    # basic length cap
    if len(q) > 2000:
        return False, "query too long"

    return True, None

# ---------- Output guard ----------
# Build simple min/max per canon_key from TRAIN
_train_df = pd.DataFrame(train_z)
_key_stats = (
    _train_df.groupby("canon_key")["y"]
    .agg(["min","max","median"])
    .rename(columns={"min":"lo","max":"hi","median":"med"})
)
_KEY_STATS = {k: {"lo": float(r["lo"]), "hi": float(r["hi"]), "med": float(r["med"])}
              for k, r in _key_stats.iterrows()}

_DEFAULT_STATS = {"lo": -1e15, "hi": 1e15, "med": 0.0}

# hints for non-negative metrics
_NONNEG_HINTS = (
    "revenue","cash","equivalents","assets","market_cap",
    "gross_profit","total_liabilities","total_assets"
)

def clamp_and_flag(k: str, y_pred: float, pad_frac: float = 0.25, enforce_nonneg: bool = False):
    """
    Clamp y_pred to [lo - pad, hi + pad] where lo/hi come from train.
    pad = pad_frac * (hi - lo), min 1.0. Flag if raw y_pred was outside.
    If y_pred is not finite, replace with median and flag.
    If enforce_nonneg=True and the item name hints non-negativity, floor at 0.
    """
    stats = _KEY_STATS.get(k, _DEFAULT_STATS)
    lo, hi, med = stats["lo"], stats["hi"], stats["med"]

    if not np.isfinite(y_pred):
        return float(med), True

    span = hi - lo
    pad = max(1.0, abs(span) * pad_frac)
    lo2, hi2 = lo - pad, hi + pad

    flagged = (y_pred < lo2) or (y_pred > hi2)
    y_safe = float(np.clip(y_pred, lo2, hi2))  # <-- define y_safe first

    # Non-negative floor for obviously non-negative items
    if enforce_nonneg and any(h in k.lower() for h in _NONNEG_HINTS):
        if y_safe < 0.0:
            y_safe = 0.0
            flagged = True  # mark as adjusted

    return y_safe, flagged

# After defining clamp_and_flag(...):
def clamp_and_flag_q(key: str, y_hat: float):
    # Delegate to the simple guardrail; keep old call-sites working
    return clamp_and_flag(key, y_hat, enforce_nonneg=True)


## 3.6.1 Evaluation: Calibration (ticker, metric) + evaluation

In [44]:
@torch.no_grad()  # inference helper: disable autograd for speed/memory
def _from_log10_signed(z):  # inverse of scaled log
    # Given z = sign(y)*log10(1 + |y|/scale), recover the signed, unscaled value:
    #   y_scaled = sign(z) * (10^|z| - 1)
    return math.copysign(10.0**abs(float(z)) - 1.0, z)

@torch.no_grad()  # end-to-end prediction from a prepared row dict
def predict_usd_from_row_embY(r: dict) -> float:
    # Prepare categorical indices (ticker/key/year) as 1-element tensors on DEVICE
    t_idx = torch.tensor([ticker2id.get(r["ticker"], 0) if TICKER_VOCAB else 0], dtype=torch.long, device=DEVICE)
    k_idx = torch.tensor([key2id[r["canon_key"]]], dtype=torch.long, device=DEVICE)
    y_idx = torch.tensor([year2id.get(int(r["year"]), 0) if YEAR_VOCAB and (r.get("year") is not None) else 0], dtype=torch.long, device=DEVICE)
    # Forward pass through the MoE regressor in z-space
    out = moe_z([r["q"]], t_idx, k_idx, y_idx, max_len=MAX_LEN)
    z_pred = float(out["z_pred"].cpu().numpy()[0])    # model output (normalized z)
    # De-normalize z: z_hat = z_pred * Z_STD + Z_MEAN
    z_hat  = z_pred * Z_STD + Z_MEAN
    # Invert the signed log transform to get scaled value
    val_scaled = _from_log10_signed(z_hat)
    # Convert back to USD via the per-row scale
    return float(val_scaled * float(r["scale"]))

def med_ratio(yhat, ygold):
    """
    Calibration helper:
    Returns median(|ygold|) / median(|yhat|), clipped to [0.5, 2.0], or None if undefined.
    Used to compute per-(ticker,metric), per-metric, and global multiplicative adjustments.
    """
    yhat = np.asarray(yhat, float); ygold = np.asarray(ygold, float)
    num = np.median(np.abs(ygold)); den = np.median(np.abs(yhat))
    if not np.isfinite(num) or not np.isfinite(den) or den == 0.0: return None
    return float(np.clip(num/den, 0.5, 2.0))

# --- Build VAL predictions for calibration ---
val_rows = []
for r in val_z:
    y_hat = predict_usd_from_row_embY(r)  # raw USD prediction
    val_rows.append({"ticker": r["ticker"], "canon_key": r["canon_key"], "y_gold": float(r["y"]), "y_hat": float(y_hat)})
df_valp = pd.DataFrame(val_rows)

# (ticker,metric) calibration → metric calibration → global ratio
# Strategy: prefer the most specific ratio available; fall back progressively.
calib_tm, calib_k = {}, {}
if not df_valp.empty:
    # Per (ticker, canon_key)
    for (t,k), g in df_valp.groupby(["ticker","canon_key"]):
        a = med_ratio(g["y_hat"], g["y_gold"])
        if a is not None: calib_tm[(t,k)] = a
    # Per canon_key only
    for k, g in df_valp.groupby("canon_key"):
        a = med_ratio(g["y_hat"], g["y_gold"])
        if a is not None: calib_k[k] = a

# Global fallback ratio
global_a = med_ratio(df_valp["y_hat"], df_valp["y_gold"]) if len(df_valp) else 1.0
if (global_a is None) or (not np.isfinite(global_a)): global_a = 1.0

def apply_calib_tm(t, k, y_hat):
    # Apply the most specific calibration available: (ticker,metric) → metric → global
    a = calib_tm.get((t,k))
    if a is None: a = calib_k.get(k, global_a)
    return float(a) * float(y_hat)

def within_tol(y_hat, y_true, tol_rel=0.02, tol_abs=5e7):
    """
    Pass/fail decision with mixed tolerance:
      - Relative tolerance: |err| <= tol_rel * max(1, |y_hat|, |y_true|)
      - Absolute floor:     |err| <= tol_abs
    """
    denom = max(1.0, abs(y_hat), abs(y_true))
    return abs(y_hat - y_true) <= max(tol_rel * denom, tol_abs)

def evaluate(pairs, k=10, tol_rel=0.02, tol_abs=5e7):
    """
    Evaluate on up to k rows:
      - Predict via model
      - Apply hierarchical calibration
      - Clamp/flag with guardrails
      - Compute abs/rel errors and pass/fail via within_tol
      - Return (DataFrame of per-row results, summary dict)
    """
    sample = pairs[:k] if len(pairs)>=k else pairs
    rows, times = [], []
    for r in sample:
        t0 = time.time()
        y_hat = predict_usd_from_row_embY(r)                          # raw model prediction (USD)
        y_hat = apply_calib_tm(r["ticker"], r["canon_key"], y_hat)    # calibrated prediction
        y_hat, flagged = clamp_and_flag(r["canon_key"], y_hat)         # output guardrail (bounds + nonneg hints)
        dt = (time.time()-t0)*1000.0
        times.append(dt)
        y = float(r["y"])
        denom = max(1.0, abs(y_hat), abs(y))
        rel_err = abs(y_hat - y) / denom
        ok = within_tol(y_hat, y, tol_rel=tol_rel, tol_abs=tol_abs)
        rows.append({"ticker": r["ticker"], "canon_key": r["canon_key"],
                     "y_gold": y, "y_pred": y_hat, "abs_err": abs(y_hat-y),
                     "rel_err": rel_err, "ok": ok, "flagged": flagged, "question": r["q"]})
    df = pd.DataFrame(rows).sort_values("rel_err").reset_index(drop=True)
    return df, {"k": len(sample), "acc": float(df["ok"].mean()), "avg_ms": float(np.mean(times))}

# --- Two evaluation regimes: strict vs practical ---
k_eval = min(9, len(test_z))
df_strict, m_strict = evaluate(test_z, k=k_eval, tol_rel=0.02, tol_abs=0)      # ±2% only (no absolute tolerance)
df_prac,   m_prac   = evaluate(test_z, k=k_eval, tol_rel=0.23, tol_abs=5e7)    # ±23% or $50M (example practical)

print("[strict  ±2%        ]", m_strict)
print("[practical ±10%|$50M]", m_prac)

# --- Pretty view of practical results (if a formatter is available) ---
if 'fmt_money' in globals():
    df_show = df_prac.copy()
    df_show["y_gold_fmt"] = df_show["y_gold"].apply(fmt_money)
    df_show["y_pred_fmt"] = df_show["y_pred"].apply(fmt_money)
    display(df_show[["ticker","canon_key","y_gold_fmt","y_pred_fmt","abs_err","rel_err","ok","flagged","question"]][:10])
else:
    display(df_prac[["ticker","canon_key","y_gold","y_pred","abs_err","rel_err","ok","flagged","question"]][:10])


[strict  ±2%        ] {'k': 9, 'acc': 0.1111111111111111, 'avg_ms': 83.14010832044814}
[practical ±10%|$50M] {'k': 9, 'acc': 0.8888888888888888, 'avg_ms': 126.89481841193304}


,ticker,canon_key,y_gold_fmt,y_pred_fmt,abs_err,rel_err,ok,flagged,question
0,ADP,revenue,$19.20 billion,$19.18 billion,1.844198e+07,0.000960,True,False,[TICKER=ADP] [YEAR=2024] [ITEM=Revenue] What w...
1,AFRM,gross profit,$1.40 billion,$1.37 billion,3.917659e+07,0.027884,True,False,[TICKER=AFRM] [YEAR=2023] [ITEM=Gross Profit] ...
2,ADP,total liabilities,$47.46 billion,$45.15 billion,2.307566e+09,0.048619,True,False,[TICKER=ADP] [YEAR=2023] [ITEM=Total Liabiliti...
3,AFRM,total liabilities,$5.62 billion,$5.99 billion,3.635686e+08,0.060747,True,False,[TICKER=AFRM] [YEAR=2023] [ITEM=Total Liabilit...
4,AFRM,net income,-$517.76 million,-$408.42 million,1.093383e+08,0.211177,True,False,[TICKER=AFRM] [YEAR=2024] [ITEM=Net Income] Wh...
5,AAGH,net income,-$761.42 thousand,-$26.43 million,2.566551e+07,0.971188,True,False,[TICKER=AAGH] [YEAR=2023] [ITEM=Net Income] Wh...
6,AFRM,cash & equivalents,$2.14 billion,$49.47 million,2.095262e+09,0.976933,False,True,"[TICKER=AFRM] [YEAR=2024] [ITEM=Cash, Cash Equ..."
7,AAGH,cost of revenue,-$124.92 thousand,-$43.16 million,4.303310e+07,0.997106,True,False,[TICKER=AAGH] [YEAR=2024] [ITEM=Cost Of Revenu...
8,AAGH,gross profit,$169.75 thousand,-$34.80 million,3.497093e+07,1.004878,True,False,[TICKER=AAGH] [YEAR=2024] [ITEM=Gross Profit] ...


## ===== Helper: compare avg speed, accuracy, and confidence (per system) =====

In [45]:
# --- Summarize and compare system speed/accuracy from a labeled results DataFrame ---
# Utilities:
# 1) compare_speed_accuracy: compute per-system aggregates (avg latency, accuracy %, confidence stats).
# 2) pairwise_table: produce a compact side-by-side comparison between two systems (e.g., RAG vs FineTuned).

def compare_speed_accuracy(df: pd.DataFrame, include_irrelevant: bool = False) -> pd.DataFrame:
    d = df.copy()
    # Optionally drop rows labeled as "irrelevant" to focus on scored items only
    if not include_irrelevant and "label" in d.columns:
        d = d[d["label"] != "irrelevant"]

    # Group by system and compute core statistics
    summary = (
        d.groupby("system", dropna=False)
         .agg(
             avg_time_s   = ("time_s", "mean"),  # mean response time in seconds
             accuracy_pct = ("correct", lambda s: float(np.mean(pd.Series(s).astype(bool)))*100.0),  # % correct
             avg_conf     = ("confidence", "mean"),  # average model-reported confidence
             conf_std     = ("confidence", "std"),   # std dev of confidence
             n            = ("correct", "size"),     # sample size
         )
         .reset_index()
    )
    # Rounding and NA handling for tidy presentation
    summary["avg_time_s"]   = summary["avg_time_s"].round(3)
    summary["accuracy_pct"] = summary["accuracy_pct"].round(1)
    summary["avg_conf"]     = summary["avg_conf"].round(3)
    summary["conf_std"]     = summary["conf_std"].fillna(0.0).round(3)
    return summary

def pairwise_table(summary: pd.DataFrame, a: str = "RAG", b: str = "FineTuned") -> pd.DataFrame | None:
    # Guardrails: need a non-empty summary with the "system" column
    if "system" not in summary or summary.empty:
        return None
    s = summary.set_index("system")
    if a not in s.index or b not in s.index:
        return None

    # Extract key metrics for the two systems
    rag_t,   ft_t   = float(s.loc[a,"avg_time_s"]), float(s.loc[b,"avg_time_s"])
    rag_acc, ft_acc = float(s.loc[a,"accuracy_pct"]), float(s.loc[b,"accuracy_pct"])
    rag_cf,  ft_cf  = float(s.loc[a,"avg_conf"]),    float(s.loc[b,"avg_conf"])

    # Compute deltas: % faster (time), percentage-point accuracy delta, and confidence delta
    ft_faster_pct = 100.0*(rag_t - ft_t)/rag_t if rag_t > 0 else float("nan")
    acc_delta_pp  = ft_acc - rag_acc
    conf_delta    = ft_cf  - rag_cf

    # Build a readable comparison table
    table = pd.DataFrame({
        "Metric":             ["Avg Time (s)", "Accuracy (%)", "Confidence (avg)"],
        a:                    [rag_t,           rag_acc,         rag_cf],
        b:                    [ft_t,            ft_acc,          ft_cf],
        "FT faster by (%)":   [round(ft_faster_pct, 1), "", ""],
        "FT - RAG (pp)":      ["", round(acc_delta_pp, 1), ""],
        "FT - RAG (Δ conf)":  ["", "", round(conf_delta, 3)],
    })
    return table


# Implement and document the advanced fine-tuning method

# Advanced Fine-Tuning: **MiniLM + Mixture-of-Experts (MoE) Adapter** (Detailed Design & Rationale)

**Goal.** We want a light-weight, data-efficient fine-tuning method that predicts *financial magnitudes* (e.g., revenue, net income) from natural-language questions with minimal overfitting and stable training on small Q/A sets. Our approach wraps a frozen **MiniLM** encoder with a tiny **Mixture-of-Experts (MoE)** adapter and structured embeddings, and learns in a *normalized target space*.

**Architecture (high-level).**  
1) **Backbone (frozen):** `sentence-transformers/all-MiniLM-L6-v2` turns the question into a vector via mean-pooling over tokens. We keep the entire encoder frozen (optionally unfreezing only the **last Transformer block**) to preserve general semantics and avoid catastrophic forgetting on small data.  
2) **Structured priors:** We learn small embeddings for `[ticker]`, `[metric (canon_key)]`, and `[year]` (dimensions 16/8/8 by default). These inject domain hints that the backbone cannot infer from text alone (e.g., *ADP vs AFRM*, 2023 vs 2024).  
3) **Projection + MoE:** We concatenate `[pooled_text, emb_ticker, emb_metric, emb_year]`, project back to `d_model`, then pass through a **MoE adapter**. Each expert is a tiny **LoRA-like** low-rank MLP (down-proj → GELU → up-proj). A learned **gating network** (two linear layers with Tanh) softly selects/weights experts per example, allowing specialization (e.g., cash-flow items vs income-statement items) **without** blowing up parameters.  
4) **Head:** A small MLP predicts a **scalar** target `z_pred`.

**Target space.** Financial values are heavy-tailed and can be negative (e.g., losses). We therefore predict a **signed-log** target:
\[
z = \operatorname{sign}(y)\,\log_{10}(1+|y|/s_\text{metric})
\]
where \( s_\text{metric} \) is a per-metric scale (e.g., billions for revenue/assets, millions for capex). We then **z-normalize** by train mean/std for stable optimization, and optimize **SmoothL1** in this space. At inference, we invert both steps to recover USD.

**Optimization & regularization.** We use **AdamW** with two parameter groups: (a) MoE/embeddings/head at `lr=2e-3`, (b) optionally unfrozen last MiniLM block at `lr=2e-4`. We clip gradients an


# 4. Testing, Evaluation & Comparison

# 4.1) Consolidated Evaluator Runner + Compact Report

**What:** One place to run RAG and FT quick checks and see metrics.

**Why:** Fast iteration loop while adjusting chunk size, CE top-k, MoE hyper-params, etc.


In [46]:
# --- Confidence penalty by label (keeps RAG lower on relevant-low) ---
# Rationale:
# - Slightly penalize confidence for queries labeled "relevant-low"
# - Heavily penalize for "irrelevant" (clamped to 0 later, kept for clarity)
LABEL_PENALTY = {
    "relevant-high": 0.00,
    "relevant-low":  0.15,
    "irrelevant":    1.00,   # we still force 0.0, but keep for clarity
}


# ----------------------------
# 0) FT confidence + wrapper
# ----------------------------
# Build training ranges once for FT confidence (if train_z exists).
# Used to boost confidence when predictions fall within seen value ranges per canon_key.
_key_ranges_ft = {}
if "train_z" in globals() and len(train_z):
    _df_tmp = pd.DataFrame(train_z)
    for k, g in _df_tmp.groupby("canon_key"):
        ys = g["y"].astype(float).values
        _key_ranges_ft[k] = (float(np.min(ys)), float(np.max(ys)))

def ft_confidence(row: dict, y_hat: float) -> float:
    """
    Heuristic confidence for fine-tuned model outputs.
    - Base 0.60, +/- 0.15 if inside/outside per-metric train range (±10% slack).
    - +0.05 for a few stable, high-signal metrics.
    - Clipped to [0.30, 0.95].
    """
    base = 0.60
    rng = _key_ranges_ft.get(row["canon_key"])
    if rng:
        lo, hi = rng
        if (y_hat >= lo - 0.1*abs(lo)) and (y_hat <= hi + 0.1*abs(hi)):
            base += 0.15
        else:
            base -= 0.15
    if row["canon_key"] in {"revenue","total assets","total liabilities","gross profit","operating cash flow"}:
        base += 0.05
    return float(np.clip(base, 0.30, 0.95))

def answer_ft(ticker: str, year: int, metric: str, raw_query: str = ""):
    """
    Uses your trained MiniLM-MoE predictor (predict_usd_from_row_embY)
    and returns a concise money string + confidence + timing.

    Returns:
      {"answer": "$X.YZ ...", "y_hat": float, "confidence": float, "secs": float}
    """
    assert "predict_usd_from_row_embY" in globals(), "predict_usd_from_row_embY missing"
    metric_l = str(metric).strip().lower()
    scale = ITEM_SCALE.get(metric_l, 1.0)     # use your per-metric scaling
    q_aug = f"[TICKER={ticker}] [YEAR={int(year)}] [ITEM={metric_l}] {raw_query}".strip()
    row = {"q": q_aug, "ticker": ticker, "year": int(year), "canon_key": metric_l, "scale": scale}
    t0 = time.time()
    y_hat = float(predict_usd_from_row_embY(row))
    secs = time.time() - t0
    conf = ft_confidence(row, y_hat)
    return {"answer": fmt_money(y_hat), "y_hat": y_hat, "confidence": conf, "secs": round(secs, 3)}

# ----------------------------
# 1) RAG-friendly validation shim
# ----------------------------
# (Lets natural questions through while keeping safety checks.)
# Uses a soft validator when strict formatting blocks are the only issue.

BANNED_PATTERNS = globals().get("BANNED_PATTERNS", [
    r"\bransomware\b", r"\bmalware\b", r"\bhack(?:ing)?\b",
])
FINANCE_KEYWORDS = globals().get("FINANCE_KEYWORDS", [
    "revenue","income","profit","loss","assets","liabilities","equity",
    "cash flow","operating cash","capex","balance sheet","income statement",
    "dividend","ebit","ebitda","net income","total assets","total liabilities"
])

def _has_fin_kw(q: str) -> bool:
    """Return True if the query contains any finance-related keyword."""
    ql = (q or "").lower()
    return any(kw in ql for kw in FINANCE_KEYWORDS)

def _is_strict_format_reason(reason: str | None) -> bool:
    """Identify reasons that are purely formatting/tag-related (eligible for soft fallback)."""
    if not reason: return False
    r = reason.lower()
    return any(p in r for p in [
        "missing finance tags", "unknown item", "unknown ticker",
        "year must be an integer", "year out of range", "query too long"
    ])

def _soft_validate_for_rag(q: str) -> dict:
    """
    Soft validator used when strict validator blocks for formatting-only reasons.
    - Blocks clearly harmful intents.
    - Allows unlabeled natural questions if they contain finance keywords.
    - Validates ticker against ALLOWED_TICKERS when present.
    """
    for pat in BANNED_PATTERNS:
        if re.search(pat, q, flags=re.I):
            return {"ok": False, "sanitized_query": q, "reason": "harmful intent", "parsed_ticker": None, "parsed_year": None}
    t, y = parse_query(q) if "parse_query" in globals() else (None, None)
    if t is not None and "ALLOWED_TICKERS" in globals() and ALLOWED_TICKERS:
        if t not in ALLOWED_TICKERS:
            return {"ok": False, "sanitized_query": q, "reason": f"unsupported ticker: {t}", "parsed_ticker": t, "parsed_year": y}
    if t is None and not _has_fin_kw(q):
        return {"ok": False, "sanitized_query": q, "reason": "no ticker or finance keyword found", "parsed_ticker": None, "parsed_year": None}
    return {"ok": True, "sanitized_query": q, "reason": None, "parsed_ticker": t, "parsed_year": y}

def validate_query_compat(query: str, mode=None):
    """
    If your strict validator blocks only for formatting/tag reasons,
    fall back to a soft RAG validator; otherwise honor the block.

    Returns dict: {ok, sanitized_query, reason, parsed_ticker, parsed_year}
    """
    try:
        res = validate_query(query, mode=mode) if mode is not None else validate_query(query)
    except TypeError:
        res = validate_query(query)

    # normalize to dict
    ok, sane, reason, pt, py = True, query, None, None, None
    if isinstance(res, dict):
        ok   = bool(res.get("ok", True))
        sane = res.get("sanitized_query") or res.get("sanitized") or query
        reason = res.get("reason")
        pt, py = res.get("parsed_ticker"), res.get("parsed_year")
    elif isinstance(res, tuple):
        ok = bool(res[0]); reason = res[1] if len(res) >= 2 else None; sane = res[2] if len(res) >= 3 and res[2] else query
    elif isinstance(res, bool):
        ok = res

    if ok:
        return {"ok": True, "sanitized_query": sane, "reason": None, "parsed_ticker": pt, "parsed_year": py}
    if _is_strict_format_reason(reason or ""):
        return _soft_validate_for_rag(sane)
    return {"ok": False, "sanitized_query": sane, "reason": reason, "parsed_ticker": pt, "parsed_year": py}

# ----------------------------
# 2) RAG confidence + numeric-only answerer
# ----------------------------
def rag_confidence(verified: dict, contexts: list, parsed: dict) -> float:
    """
    Heuristic confidence for RAG answers based on:
      - Top reranker prob, mean of top-3 probs
      - Context support for parsed ticker/year
      - Structured lookup success (bonus)
      - Penalty if ticker/year missing
    Clipped to [0.05, 0.98].
    """
    probs = [float(c.get("rerank_prob", 0.5)) for c in (contexts or [])[:5]]
    top_p   = max(probs) if probs else 0.0
    mean_p3 = sum(probs[:3]) / max(1, len(probs[:3]))
    pt, py = parsed.get("parsed_ticker"), parsed.get("parsed_year")
    support = 0
    for c in (contexts or [])[:5]:
        ok_t = (pt is None) or (c.get("ticker") == pt)
        ok_y = (py is None) or (int(c.get("year", -1)) == int(py))
        if ok_t and ok_y:
            support += 1
    support_frac = support / max(1, min(5, len(contexts or [])))
    structured_ok = (verified.get("source") == "structured_lookup") and (not verified.get("flagged", False))
    base = 0.35 + 0.30*top_p + 0.20*mean_p3 + 0.15*support_frac
    if structured_ok: base += 0.10
    if pt is None or py is None: base -= 0.10
    return float(np.clip(base, 0.05, 0.98))

def answer_query(query: str, k_ctx: int = 5, alpha: float = 0.6, numeric_only: bool = True) -> dict:
    """
    If structured lookup succeeds → return just the money string (e.g., "$19.20 billion").
    Else return a short RAG summary with citations.
    """
    ticker, year = parse_query(query) if "parse_query" in globals() else (None, None)
    metric = detect_metric(query)

    direct = None
    if ticker and year:
        # Attempt structured lookup via metric name, else fuzzy line-item extraction
        if metric:
            direct = lookup_metric(df_fin, ticker, year, metric)
        if direct is None:
            li = extract_line_item_from_question(query, ticker, year, df_fin)
            if li:
                sub = df_fin.loc[
                    (df_fin["ticker"]==ticker) & (df_fin["fiscal_year"]==year) &
                    (df_fin["line_item"].str.contains(re.escape(li), case=False, regex=True))
                ].copy()
                if not sub.empty:
                    row = (sub.assign(abs_val=sub["value"].abs()).nlargest(1, "abs_val").iloc[0])
                    v_norm, u_norm = normalize_value(row["value"], row["unit"])
                    direct = {"line_item": row["line_item"], "value_norm": v_norm, "unit_norm": u_norm, "statement": row["statement"]}

    # Retrieve contexts regardless, for citations/backup summary
    ctxs = retrieve_contexts(query, k_ctx=k_ctx, alpha=alpha)

    if direct:
        # Structured numeric answer
        val_str = fmt_money(direct["value_norm"])
        answer_text = val_str if numeric_only else f"{ticker} {year} {(metric.title() if metric else direct['line_item'])}: {val_str} ({direct['unit_norm']})."
        return {
            "answer": answer_text,
            "source": "structured_lookup",
            "ticker": ticker, "year": year, "metric": (metric or direct["line_item"]),
            "value_norm": direct["value_norm"], "unit_norm": direct["unit_norm"],
            "statement": direct["statement"], "citations": citations_from_contexts(ctxs)
        }

    # RAG fallback summary (first line of up to 3 contexts)
    bullets = []
    for c in ctxs[:min(3, len(ctxs))]:
        txt = c["text"].strip().split("\n")[0]
        bullets.append(f"• {txt[:220]}{'…' if len(txt)>220 else ''}")
    return {
        "answer": "RAG summary:\n" + "\n".join(bullets) if bullets else "RAG: no contexts.",
        "source": "rag_fallback",
        "ticker": ticker, "year": year, "metric": metric,
        "citations": citations_from_contexts(ctxs)
    }

def answer_query_guarded(query: str, k_ctx: int = 5, alpha: float = 0.6, mode=None) -> dict:
    """
    Wrapper that:
      1) Soft/strict-validates the query (safety + format).
      2) Runs retrieval + answering.
      3) Verifies and assigns a RAG confidence.
      4) Ensures citations are attached.
    """
    v = validate_query_compat(query, mode=mode)
    if not v.get("ok", True):
        return {"blocked": True, "reason": v.get("reason"), "sanitized_query": v.get("sanitized_query", query)}
    q_san = v.get("sanitized_query", query)
    ctxs  = retrieve_contexts(q_san, k_ctx=k_ctx, alpha=alpha)
    ans   = answer_query(q_san, k_ctx=k_ctx, alpha=alpha)
    verified = verify_output(ans, tol=0.02) if "verify_output" in globals() else dict(ans, flagged=False, confidence=0.5)
    verified["confidence"] = rag_confidence(verified, ctxs, v)
    if not verified.get("citations"):
        verified["citations"] = [os.path.basename(c["source"]) for c in ctxs[:3] if c.get("source")]
    return {
        "blocked": False,
        "query_sanitized": q_san,
        "parsed_ticker": v.get("parsed_ticker"),
        "parsed_year": v.get("parsed_year"),
        "result": verified,
    }

# ----------------------------
# 3) Natural question selection (from corpus if possible)
# ----------------------------
def _load_qa_pairs_from_disk():
    """Load QA pairs from disk if not present in memory (used to craft natural questions)."""
    p = "out/qa_pairs.jsonl"
    if os.path.exists(p):
        with open(p, "r", encoding="utf-8") as f:
            return [json.loads(line) for line in f]
    return []

_QA = qa_pairs if "qa_pairs" in globals() else _load_qa_pairs_from_disk()

def _matches_metric(line_item: str, metric: str) -> bool:
    """Loose match of a QA line_item to the requested metric using predefined patterns and fallback exact word."""
    pats = METRIC_PATTERNS.get(metric, [])
    li = str(line_item or "")
    for pat in pats:
        if re.search(pat, li, flags=re.I):
            return True
    if metric and re.search(rf"\b{re.escape(metric)}\b", li, flags=re.I):
        return True
    return False

def question_from_corpus(ticker: str, year: int, metric: str) -> str | None:
    """Search QA corpus for a natural-sounding question matching (ticker, year, metric)."""
    if not _QA: return None
    cands = [q for q in _QA if q.get("ticker")==ticker and int(q.get("year", -1))==int(year)]
    cands = [q for q in cands if _matches_metric(q.get("line_item",""), metric)]
    if cands:
        return min(cands, key=lambda r: len(r.get("question","") or "~")).get("question")
    return None

def question_template(ticker: str, year: int, metric: str) -> str:
    """Deterministic template fallback to phrase the question naturally."""
    m = (metric or "").strip().lower()
    m = {"cash & equivalents": "cash and cash equivalents"}.get(m, m)
    return f"What was {ticker}'s {m} in {int(year)}?"

def make_natural_question(ticker: str, year: int, metric: str) -> str:
    """Prefer corpus question if available; otherwise use the template."""
    return question_from_corpus(ticker, year, metric) or question_template(ticker, year, metric)

# ----------------------------
# 4) 4.1 Official evaluation runner
# ----------------------------
official = [
    {"label": "relevant-high", "ticker": "ADP",  "year": 2024, "metric": "revenue",     "query": "ADP 2024 revenue"},
    {"label": "relevant-low",  "ticker": "AAGH", "year": 2023, "metric": "net income",  "query": "AAGH 2023 net income"},
    {"label": "irrelevant",    "ticker": None,   "year": None, "metric": None,          "query": "What is the capital of France?"},
]

def apply_label_penalty(conf: float, label: str) -> float:
    """Subtract label-dependent penalty and clip to [0.0, 0.98]."""
    return float(np.clip(float(conf) - float(LABEL_PENALTY.get(label, 0.0)), 0.0, 0.98))

# --- ground-truth helper (handles (val, unit) or plain float)
def _gt_val_only(t, y, m):
    """Return ground-truth numeric value only (normalized), or None if not found."""
    if not (t and y and m): return None
    got = ground_truth_value(df_fin, t, int(y), m)
    if got is None: return None
    if isinstance(got, (tuple, list)): return float(got[0])
    return float(got)

# --- main runner
def run_official(official,
                 rag_tol_rel=0.02, rag_tol_abs=0.0,      # strict RAG
                 ft_tol_rel=0.23, ft_tol_abs=5e7):       # practical FT
    """
    Evaluate both pipelines on an official set:
      - RAG: guarded answering with confidence & penalties
      - FT : fine-tuned numeric model with timing and accuracy

    Returns:
      pd.DataFrame with columns: system, label, query, answer, confidence, time_s, correct
    """
    rows = []
    for q in official:
        # Natural question unless irrelevant
        q_txt = q["query"] if q["label"]=="irrelevant" else make_natural_question(
            q["ticker"], int(q["year"]), q["metric"]
        )

        # ---------- RAG ----------
        t0 = time.time()
        rag = answer_query_guarded(q_txt, k_ctx=5, alpha=0.6)
        rag_dt = time.time() - t0

        if rag.get("blocked") or q["label"] == "irrelevant":
            rag_ans, rag_conf, rag_ok = "Data not in scope", 0.0, False
        else:
            r = rag["result"]
            rag_ans  = r.get("answer", "")
            rag_conf = float(r.get("confidence") or 0.0)
            rag_conf = apply_label_penalty(rag_conf, q["label"])

            gt = _gt_val_only(q.get("ticker"), q.get("year"), q.get("metric"))
            if gt is None:
                rag_ok = False
            else:
                pred = r.get("value_norm")
                if pred is None:
                    # parse "$X [thousand/million/billion/trillion]" if needed
                    m = re.search(r"\$([0-9]+(?:\.[0-9]+)?)\s*(thousand|million|billion|trillion)?", rag_ans, flags=re.I)
                    if m:
                        mul = {"thousand":1e3,"million":1e6,"billion":1e9,"trillion":1e12}.get((m.group(2) or "").lower(), 1.0)
                        pred = float(m.group(1)) * mul
                denom = max(1.0, abs(pred or 0.0), abs(gt))
                rag_ok = (pred is not None) and (abs((pred or 0) - gt) <= max(rag_tol_rel*denom, rag_tol_abs))

        rows.append({
            "system": "RAG",
            "label": q["label"],
            "query": q_txt,
            "answer": rag_ans,
            "confidence": float(rag_conf),
            "time_s": round(rag_dt, 2),
            "correct": bool(rag_ok),
        })

        # ---------- Fine-Tune ----------
        t0 = time.time()
        if q["label"] == "irrelevant":
            ft_ans, ft_conf, ft_ok, ft_dt = "Not applicable", 0.0, False, 0.0
        else:
            out   = answer_ft(q.get("ticker"), int(q.get("year")), q.get("metric"), raw_query=q_txt)
            ft_dt = time.time() - t0
            ft_ans, ft_conf = out["answer"], float(out.get("confidence", 0.0))

            gt = _gt_val_only(q.get("ticker"), q.get("year"), q.get("metric"))
            if gt is None:
                ft_ok = False
            else:
                denom = max(1.0, abs(out["y_hat"]), abs(gt))
                ft_ok = abs(out["y_hat"] - gt) <= max(ft_tol_rel*denom, ft_tol_abs)

        rows.append({
            "system": "Fine-Tune",   # pretty name for report
            "label": q["label"],
            "query": q_txt,
            "answer": ft_ans,
            "confidence": float(ft_conf),
            "time_s": round(ft_dt, 2),
            "correct": bool(ft_ok),
        })

    return pd.DataFrame(rows)

# --- compact report (matches your screenshot columns)
def to_report(df: pd.DataFrame) -> pd.DataFrame:
    """Reduce the raw results to a concise, presentation-friendly view."""
    view = pd.DataFrame({
        "Question": df["query"],
        "Method":   df["system"],
        "Answer":   df["answer"],
        "Confidence": df["confidence"].astype(float).round(2),
        "Time (s)":   df["time_s"].astype(float).round(2),
        "Correct (Y/N)": df["correct"].map(lambda b: "Y" if bool(b) else "N"),
    })
    return view

# ---------- Example official set (edit as you like) ----------
official = [
    {"label": "relevant-high", "ticker": "ADP",  "year": 2024, "metric": "revenue",     "query": "ADP 2024 revenue"},
    {"label": "relevant-low",  "ticker": "AAGH", "year": 2023, "metric": "net income",  "query": "AAGH 2023 net income"},
    {"label": "irrelevant",    "ticker": None,   "year": None, "metric": None,          "query": "What is the capital of France?"},
]

# ---------- Run + display ----------
df_official = run_official(official, ft_tol_rel=0.23, ft_tol_abs=5e7)
display(to_report(df_official))

# (optional) save to disk for later review
to_report(df_official).to_csv("out/report_view.csv", index=False)

# ===== 4.1 summary =====
# Aggregated per-system stats (excluding irrelevant by default)
summary_41 = compare_speed_accuracy(df_official, include_irrelevant=False)
display(summary_41)       # columns: system, avg_time_s, accuracy_pct, avg_conf, conf_std, n


,Question,Method,Answer,Confidence,Time (s),Correct (Y/N)
0,What was ADP's Revenue in 2024?,RAG,$19.20 billion,0.98,1.16,Y
1,What was ADP's Revenue in 2024?,Fine-Tune,$19.51 billion,0.80,0.03,Y
2,What was AAGH's Net Income in 2023?,RAG,-$761.42 thousand,0.83,1.20,Y
3,What was AAGH's Net Income in 2023?,Fine-Tune,-$52.85 million,0.75,0.03,N
4,What is the capital of France?,RAG,Data not in scope,0.00,0.00,N
5,What is the capital of France?,Fine-Tune,Not applicable,0.00,0.00,N


,system,avg_time_s,accuracy_pct,avg_conf,conf_std,n
0,Fine-Tune,0.03,50.0,0.775,0.035,2
1,RAG,1.18,100.0,0.905,0.106,2


# 4.2) Side-by-Side Demo: **RAG vs FT vs CE** (Interactive)

**What (extra detail across RAG/CE/FT/MoE):**
- **RAG**: hybrid retrieval (dense+TF-IDF) → **CE** reranker → *summary or exact* answer if lookup succeeded.  
- **FT (MoE)**: directly **predicts numbers** from the question text + (ticker/metric/year) embeddings; great when structured data is consistent.  
- Use this cell to compare per-query behavior and see where CE reranking helps RAG vs where MoE shines.



In [47]:
# ---- Prelude: unseen canon_key fallback for FT predictor (prevents KeyError) ----
# Ensure all required globals from earlier steps are available before continuing.
# These assertions guard against calling inference without the proper setup.
assert 'key2id' in globals() and len(key2id) > 0, "Load vocabs before 4.2"
assert 'moe_z' in globals(), "moe_z must be created/loaded before 4.2"
assert 'Z_MEAN' in globals() and 'Z_STD' in globals(), "z-norm stats required"
assert 'MAX_LEN' in globals() and 'DEVICE' in globals(), "inference config required"



# z normalization (train only)
# Recompute normalization stats for z (signed log-scaled target) from training split.
# Note: This duplicates earlier logic intentionally to make this cell standalone.
z_train = np.array([r["z"] for r in train_z], dtype=np.float32)
Z_MEAN = float(z_train.mean()) if len(z_train) else 0.0
Z_STD  = float(z_train.std())  if len(z_train) > 1 else 1.0
if Z_STD == 0.0: Z_STD = 1.0
print(f"[z-norm] mean={Z_MEAN:.4f} std={Z_STD:.4f}")

# vocabularies
# Build/refresh categorical vocabularies across all splits for robust inference.
TICKER_VOCAB = sorted({r["ticker"] for r in (train_z+val_z+test_z) if r.get("ticker")})
KEY_VOCAB    = sorted({r["canon_key"] for r in (train_z+val_z+test_z)})
YEAR_VOCAB   = sorted({int(r["year"]) for r in (train_z+val_z+test_z) if r.get("year") is not None})

# Categorical -> index mappings used by embedding layers.
ticker2id = {t:i for i,t in enumerate(TICKER_VOCAB)}
key2id    = {k:i for i,k in enumerate(KEY_VOCAB)}
year2id   = {y:i for i,y in enumerate(YEAR_VOCAB)}



# Fallback canonical key if an unseen canon_key shows up at inference time.
# Prefer "revenue" if present; otherwise pick the first available key.
FALLBACK_KEY = "revenue" if ("revenue" in key2id) else next(iter(key2id.keys()))
_UNKNOWN_KEYS_SEEN = set()  # for telemetry/debugging (tracks unseen keys encountered)

def _get_key_id_safe(canon_key: str) -> int:
    """
    Robustly map a canonical key string to its index:
    - Known key → return its index.
    - Unknown key → record and return FALLBACK_KEY index (prevents KeyError).
    """
    k = (canon_key or "").strip().lower()
    kid = key2id.get(k)
    if kid is None:
        if k not in _UNKNOWN_KEYS_SEEN:
            _UNKNOWN_KEYS_SEEN.add(k)
        kid = key2id[FALLBACK_KEY]
    return int(kid)

@torch.no_grad()
def predict_usd_from_row_embY(r: dict) -> float:
    """
    r expects: {'q', 'ticker', 'year', 'canon_key', 'scale'}
    Returns: predicted USD float.

    Steps:
      1) Convert categorical fields to indices (with safe fallback for canon_key).
      2) Forward through MoE z-regressor to get normalized z.
      3) De-normalize z, invert signed log10 transform.
      4) Multiply by per-metric scale to return USD.
    """
    # indices
    t_idx = torch.tensor([ticker2id.get(r.get("ticker"), 0) if 'TICKER_VOCAB' in globals() and TICKER_VOCAB else 0],
                         dtype=torch.long, device=DEVICE)
    k_idx = torch.tensor([_get_key_id_safe(r.get("canon_key", ""))],
                         dtype=torch.long, device=DEVICE)
    y_raw = r.get("year")
    y_idx = torch.tensor([year2id.get(int(y_raw), 0) if ('YEAR_VOCAB' in globals() and YEAR_VOCAB and y_raw is not None) else 0],
                         dtype=torch.long, device=DEVICE)

    # forward
    out = moe_z([r.get("q","")], t_idx, k_idx, y_idx, max_len=MAX_LEN)
    z_norm_pred = out["z_pred"].view(-1)[0].detach().cpu().item()

    # de-normalize & invert signed log
    z_pred = z_norm_pred * float(Z_STD) + float(Z_MEAN)
    # inverse of sign*log10(1 + |y|/scale) → sign(z)*(10^|z| - 1)
    y_unscaled = math.copysign(10.0**abs(float(z_pred)) - 1.0, z_pred)

    # apply per-metric scale from upstream (brings back to USD or item units)
    return float(y_unscaled * float(r.get("scale", 1.0)))


# ---- Helpers reused from earlier steps (must exist): fmt_money, answer_query_guarded, answer_ft, extract_numeric_from_text, make_natural_question, apply_label_penalty ----
# Assumptions:
#  - fmt_money(amount_float) -> "$X unit"
#  - answer_query_guarded(q, k_ctx, alpha) returns dict with result.answer, result.value_norm, result.confidence
#  - answer_ft(ticker, year, metric, raw_query) returns {"answer": str, "y_hat": float, "confidence": float, "secs": float}
#  - extract_numeric_from_text(s) returns base-USD float or None
#  - make_natural_question(ticker, year, metric) returns NL question text
#  - apply_label_penalty(conf, label) optional; if missing we no-op

def _gt_val_only(t, y, m):
    """Return ground-truth numeric (USD). Compatible with both (val,unit) and val returns."""
    if not (t and y and m): return None
    got = ground_truth_value(df_fin, t, int(y), m)
    if got is None: return None
    if isinstance(got, (tuple, list)): return float(got[0])
    return float(got)

# ---- Config: FT correctness thresholds (as requested) ----
# Practical tolerance for FT evaluation: relative ±23% OR absolute $50M
FT_TOL_REL = 0.23     # ±23%
FT_TOL_ABS = 5e7      # or $50M absolute

# ---- Build at least 10 evaluation questions ----
# Mix of high/low relevance and a couple of irrelevant controls for completeness.
def build_ext_questions():
    qs = [
        # High relevance / larger caps
        {"label":"relevant-high","ticker":"ADP",  "year":2024, "metric":"revenue"},
        {"label":"relevant-high","ticker":"ADP",  "year":2023, "metric":"gross profit"},
        {"label":"relevant-high","ticker":"AFRM", "year":2024, "metric":"total liabilities"},
        {"label":"relevant-high","ticker":"AEHR", "year":2024, "metric":"revenue"},

        # Lower relevance / noisier items or smaller magnitudes
        {"label":"relevant-low", "ticker":"AAGH", "year":2023, "metric":"net income"},
        {"label":"relevant-low", "ticker":"AFRM", "year":2023, "metric":"operating cash flow"},
        {"label":"relevant-low", "ticker":"ADP",  "year":2023, "metric":"shareholders equity"},
        {"label":"relevant-low", "ticker":"AEHR", "year":2023, "metric":"gross profit"},

        # A couple more mixed
        {"label":"relevant-high","ticker":"AFRM", "year":2024, "metric":"revenue"},
        {"label":"relevant-low", "ticker":"ADP",  "year":2024, "metric":"total assets"},

        # Irrelevant controls
        {"label":"irrelevant","ticker":None,"year":None,"metric":None,"query":"What is the capital of France?"},
        {"label":"irrelevant","ticker":None,"year":None,"metric":None,"query":"Who won the 2018 FIFA World Cup?"},
    ]
    return qs

def _make_query_text(q):
    """
    Produce the natural-language question used for evaluation:
    - Use corpus/template for relevant items.
    - Pass through the free-form text for irrelevant controls.
    """
    if q["label"] != "irrelevant":
        if "make_natural_question" in globals():
            return make_natural_question(q["ticker"], int(q["year"]), q["metric"])
        else:
            m = q["metric"]
            return f"What was {q['ticker']}'s {m} in {int(q['year'])}?"
    return q.get("query","")

def run_extended(questions, show=False):
    """
    Evaluate RAG and FineTuned systems on the provided question set.
    - RAG: strict ±2% check (numeric-only) against ground truth.
    - FT : practical tolerance (±23% or $50M).
    Returns a DataFrame with per-question results.
    """
    rows = []
    for q in questions:
        q_txt = _make_query_text(q)

        # ---------- RAG ----------
        t0 = time.time()
        rag = answer_query_guarded(q_txt, k_ctx=5, alpha=0.6)
        rag_dt = time.time() - t0

        if rag.get("blocked") or q["label"] == "irrelevant":
            rag_ans, rag_conf, rag_ok = "Data not in scope", 0.0, False
        else:
            r = rag["result"]
            rag_ans  = r.get("answer","")
            rag_conf = float(r.get("confidence") or 0.0)
            if "apply_label_penalty" in globals():
                rag_conf = apply_label_penalty(rag_conf, q["label"])
            gt = _gt_val_only(q.get("ticker"), q.get("year"), q.get("metric"))
            if gt is None:
                rag_ok = False
            else:
                pred = r.get("value_norm")
                if pred is None and 'extract_numeric_from_text' in globals():
                    # Fallback: parse numeric value from textual answer
                    pred = extract_numeric_from_text(rag_ans)
                denom = max(1.0, abs(pred or 0.0), abs(gt))
                rag_ok = (pred is not None) and (abs((pred or 0) - gt)/denom <= 0.02)  # strict ±2%

        rows.append({
            "system": "RAG",
            "label": q["label"],
            "query": q_txt,
            "ticker": q.get("ticker"),
            "year": float(q.get("year")) if q.get("year") else float("nan"),
            "metric": q.get("metric"),
            "ground_truth": fmt_money(_gt_val_only(q.get("ticker"), q.get("year"), q.get("metric"))) if q["label"]!="irrelevant" else "-",
            "answer": rag_ans,
            "confidence": float(rag_conf),
            "time_s": round(rag_dt, 3),
            "correct": bool(rag_ok),
        })

        # ---------- FineTuned ----------
        t0 = time.time()
        if q["label"] == "irrelevant":
            # For irrelevant controls, FT is not applicable by design
            ft_ans, ft_conf, ft_ok, ft_dt = "Not applicable", 0.0, False, 0.0
        else:
            out   = answer_ft(q.get("ticker"), int(q.get("year")), q.get("metric"), raw_query=q_txt)
            ft_dt = time.time() - t0
            ft_ans, ft_conf = out["answer"], float(out.get("confidence", 0.0))
            gt = _gt_val_only(q.get("ticker"), q.get("year"), q.get("metric"))
            if gt is None:
                ft_ok = False
            else:
                denom = max(1.0, abs(out["y_hat"]), abs(gt))
                ft_ok = abs(out["y_hat"] - gt) <= max(FT_TOL_REL * denom, FT_TOL_ABS)

        rows.append({
            "system": "FineTuned",
            "label": q["label"],
            "query": q_txt,
            "ticker": q.get("ticker"),
            "year": float(q.get("year")) if q.get("year") else float("nan"),
            "metric": q.get("metric"),
            "ground_truth": fmt_money(_gt_val_only(q.get("ticker"), q.get("year"), q.get("metric"))) if q["label"]!="irrelevant" else "-",
            "answer": ft_ans,
            "confidence": float(ft_conf),
            "time_s": round(ft_dt, 3),
            "correct": bool(ft_ok),
        })

    cols = ["system","label","query","ticker","year","metric","ground_truth","answer","confidence","time_s","correct"]
    df = pd.DataFrame(rows)[cols]

    # No automatic display here; only show if explicitly requested.
    if show:
        display(df)  # plain DataFrame, no Styler

    return df

# ---- Run 4.2 ----
ext_questions = build_ext_questions()
df_extended = run_extended(ext_questions)

# Keep only the requested columns, rename, order, and convert True/False -> Y/N
_cols_map_ext = {
    "query":        "Question",
    "system":       "Method",
    "ground_truth": "Real Answer",
    "answer":       "Answer",
    "confidence":   "Confidence",
    "time_s":       "Time (s)",
    "correct":      "Correct (Y/N)",
}
_order_ext = ["Question", "Method", "Real Answer", "Answer", "Confidence", "Time (s)", "Correct (Y/N)"]

df_extended_view = (
    df_extended
      .rename(columns=_cols_map_ext)
      .loc[:, _order_ext]
      .assign(**{"Correct (Y/N)": lambda d: d["Correct (Y/N)"].map({True: "Y", False: "N"})})
    # .assign(Confidence=lambda d: d["Confidence"].round(3),  # <- optional rounding
    #         **{"Time (s)": lambda d: d["Time (s)"].round(3)})
)

# ---- pretty display: wrap long questions + show 2 decimals for numeric cols ----
_question_width = 320  # px, tweak as you like

styler = (
    df_extended_view.style
        .format({"Confidence": "{:.2f}", "Time (s)": "{:.2f}"})   # fixed 2 d.p.
        .set_table_styles([
            {"selector": "table", "props": [("table-layout", "fixed"), ("width", "100%")]},
            {"selector": "th, td", "props": [("vertical-align", "top"), ("text-align", "left")]},
        ])
        .set_properties(
            subset=["Question"],
            **{
                "max-width": f"{_question_width}px",
                "width": f"{_question_width}px",
                "white-space": "normal",
                "overflow-wrap": "anywhere",
                "word-break": "break-word",
            },
        )
)

# Display the nicely formatted table (Jupyter-friendly)
display(styler)



# ===== 4.2 summary =====
# Aggregate per-system speed/accuracy (exclude irrelevant by default).
summary_42 = compare_speed_accuracy(df_extended, include_irrelevant=False)
display(summary_42)

# Optional: side-by-side comparison between RAG and FineTuned if both present.
pair_42 = pairwise_table(summary_42)
if pair_42 is not None:
    display(pair_42)


[z-norm] mean=0.3043 std=0.6026


,Question,Method,Real Answer,Answer,Confidence,Time (s),Correct (Y/N)
0,What was ADP's Revenue in 2024?,RAG,$19.20 billion,$19.20 billion,0.98,1.16,Y
1,What was ADP's Revenue in 2024?,FineTuned,$19.20 billion,$19.51 billion,0.80,0.05,Y
2,What was ADP's Gross Profit in 2023?,RAG,$9.35 billion,$9.35 billion,0.98,1.12,Y
3,What was ADP's Gross Profit in 2023?,FineTuned,$9.35 billion,$8.12 billion,0.80,0.03,Y
4,What was AFRM's Total Liabilities in 2024?,RAG,$6.79 billion,$6.79 billion,0.98,1.32,Y
5,What was AFRM's Total Liabilities in 2024?,FineTuned,$6.79 billion,$7.04 billion,0.80,0.03,Y
6,What was AEHR's Revenue in 2024?,RAG,$66.22 million,$66.22 million,0.98,1.04,Y
7,What was AEHR's Revenue in 2024?,FineTuned,$66.22 million,$21.50 million,0.80,0.03,Y
8,What was AAGH's Net Income in 2023?,RAG,-$761.42 thousand,-$761.42 thousand,0.83,1.19,Y
9,What was AAGH's Net Income in 2023?,FineTuned,-$761.42 thousand,-$52.85 million,0.75,0.03,N


,system,avg_time_s,accuracy_pct,avg_conf,conf_std,n
0,FineTuned,0.034,70.0,0.730,0.109,10
1,RAG,1.206,100.0,0.905,0.079,10


,Metric,RAG,FineTuned,FT faster by (%),FT - RAG (pp),FT - RAG (Δ conf)
0,Avg Time (s),1.206,0.034,97.2,,
1,Accuracy (%),100.000,70.000,,-30.0,
2,Confidence (avg),0.905,0.730,,,-0.175
